## Prerequisite

In [ ]:
!pip install -q "numpy<2.0" opencv-python-headless ultralytics huggingface_hub matplotlib scipy roboflow supervision chess

In [ ]:
import cv2
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
import os
from collections import deque,defaultdict,Counter
from PIL import Image
import numpy as np, math
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from typing import Dict, Any
from kaggle_secrets import UserSecretsClient
from roboflow import Roboflow
import shutil
import chess
import chess.pgn
import collections
from sklearn.cluster import KMeans

In [ ]:
!wget https://github.com/Ak1ralin/Digital_Image/raw/refs/heads/main/Final_project/yolov8n.pt #download hand detection model
!wget https://github.com/Ak1ralin/Digital_Image/raw/refs/heads/main/Final_project/yolo8m-largeData.pt #download chess detection model

In [ ]:
# use 'yolov8 nano' for hand detection
hand_model = YOLO('yolov8n.pt')
# use self-trained yolo8 model for chess piece detection
chess_model = YOLO("yolo8m-largeData.pt")

## Illustration

In [ ]:
def print_board(board):
    print("="*60)
    for row in board:
        print("  ".join(map(str, row)))
    print("="*60)

In [ ]:
def show_image(img, title="", convertWith=None): # convertWith ex. cv2.COLOR_BGR2RGB
    final_img = cv2.cvtColor(img, convertWith) if convertWith is not None else img
    plt.figure(figsize=(5, 5)) # กำหนดขนาดกลางๆ ไว้ก่อน
    plt.imshow(final_img)
    plt.title(title)
    plt.axis('off') 
    plt.show()

## Video2Image

In [ ]:
def video2image(video_path, hand_model, output_folder="clean_frame", debug=True):
    os.makedirs(output_folder, exist_ok=True) #folder too save image
    
    cap = cv2.VideoCapture(video_path) #Open video
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    # --- SETTINGS ---
    FRAME_SKIP = 15       # 15 frame -> 1 image
    CONFIDENCE = 0.125    # be 12.5% sure it's a hand -> blocked, make it sensitive this just 1st step to filter hand
    BUFFER_SIZE = 2       # Must see same state 2 times to switch -> avoid noise
    
    # --- STATE VARIABLES ---
    frame_count = 0
    chunk_counter = 1     
    curr_state = None  
    
    # buffer to store recent results for check consecutive states 
    result_buffer = deque(maxlen=BUFFER_SIZE)  # save as tuple (frame_image, frame_number, state_string)

    print(f"Processing... Saving ordered chunks to '/{output_folder}'")

    while True:
        ret, frame = cap.read()
        if not ret: # have next frame or not
            break
        
        frame_count += 1
        if frame_count % FRAME_SKIP != 0:
            continue  # skip frames 

        # 1. DETECT HAND / BLOCK
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)

        results_list = hand_model(
            [pil_img],          
            verbose=False,
            conf=CONFIDENCE,
            classes=[0],       # class 0 = hand (in your small YOLO)
        )

        result = results_list[0]
        is_blocked = len(result.boxes) > 0
        frame_state = "blocked" if is_blocked else "clean"
        
        # 2. ADD TO BUFFER
        result_buffer.append((frame.copy(), frame_count, frame_state))
        
        if len(result_buffer) < BUFFER_SIZE:
            continue

        # 3. CHECK FOR CONSENSUS
        recent_states = [x[2] for x in result_buffer]
        if not all(s == recent_states[0] for s in recent_states):
            continue

        stable_state = recent_states[0]
            
        # --- SCENARIO A: FIRST CHUNK EVER ---
        if curr_state is None:
            curr_state = stable_state
            img, num, state = result_buffer[0]
            filename = f"{output_folder}/{chunk_counter:04d}_frame{num}_START_{state}.jpg"
            cv2.imwrite(filename, img)
            print(f"[{chunk_counter:03d}] Started {state} at frame {num}")

        # --- SCENARIO B: STATE SWITCH ---
        elif curr_state != stable_state:
            # 1. Close OLD chunk
            if debug:
                img_end, num_end, _ = result_buffer[0]
                filename_end = f"{output_folder}/{chunk_counter:04d}_frame{num_end}_END_{curr_state}.jpg"
                cv2.imwrite(filename_end, img_end)
            
            # 2. Start NEW chunk
            chunk_counter += 1
            curr_state = stable_state
            
            img_start, num_start, state_start = result_buffer[-1]
            if state_start == "clean" or debug:
                filename_start = f"{output_folder}/{chunk_counter:04d}_frame{num_start}_START_{state_start}.jpg"
                cv2.imwrite(filename_start, img_start)
            
            print(f"[{chunk_counter:03d}] Switched to {state_start} at frame {num_start}")

    # --- CLOSE LAST CHUNK ---
    if len(result_buffer) > 0 and curr_state is not None and debug:
        img_last, num_last, _ = result_buffer[-1]
        filename_final = f"{output_folder}/{chunk_counter:04d}_frame{num_last}_END_{curr_state}.jpg"
        cv2.imwrite(filename_final, img_last)

    cap.release()
    print("Video2Image processing complete.")

## Get ROI

In [ ]:
# Parameters 
MAX_W = 640

# Preprocess
GAUSS_K = (5,5)
CANNY_LOW, CANNY_HIGH = 70, 170
DILATE_K, ERODE_K = (5,5), (5,5)

# Long-line anchors (for clustering only)
MIN_LEN_RATIO = 0.05

# Join/merge params (scale with image diagonal)
ANGLE_TOL_DEG      = 5.0
RHO_EPS_RATIO      = 0.020
GAP_EPS_RATIO      = 0.010
POST_MIN_LEN_RATIO = 0.15

# Inner mask (optional)
MASK_ERODE_RATIO_INNER = 0.15

# Texture heuristic
XSEC_HALF_PIX   = 6
XSEC_NSAMPLES   = 12

# Quality mix
ANG_NORM_DEG       = 5.0
BETA_ANG           = 0.70
BETA_LEN0          = 0.15
BETA_LEN1          = 0.85
TEXT_BONUS         = 0.9

# Selection weights & guards
W_RHO = 3.0        
W_Q   = 0.15   
INNER_BAND_MARGIN = 0.20  
FAR_GATE_MULT     = 2.0  
MIN_SEP_MULT      = 0.45 
LOW_SEP_MULT      = 1.00  
HIGH_SEP_MULT     = 1.50  
MIN_COVER         = 0.35 

# EXACT number of lines per family
N_GRID = 9

# Debug
OUT = Path("/kaggle/working/vanish9x9_from_temp")

In [ ]:
# Helper Functions

def preprocess_edges(bgr):
    g = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.GaussianBlur(g, GAUSS_K, 0)
    e = cv2.Canny(g, CANNY_LOW, CANNY_HIGH)
    e = cv2.dilate(e, cv2.getStructuringElement(cv2.MORPH_RECT, DILATE_K), 1)
    e = cv2.erode( e, cv2.getStructuringElement(cv2.MORPH_RECT, ERODE_K), 1)
    return e

def erode_mask_by_ratio(mask, ratio):
    h,w = mask.shape[:2]
    k = max(3, int(ratio * min(h,w)))
    if k % 2 == 0: k += 1
    return cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k,k)), 1)

def lsd_segments(gray, mask=None, min_len_ratio=0.0):
    h, w = gray.shape[:2]
    diag = math.hypot(w, h)
    min_len = max(0.0, float(min_len_ratio) * diag)
    lsd = cv2.createLineSegmentDetector(cv2.LSD_REFINE_ADV)
    g = gray if mask is None else cv2.bitwise_and(gray, gray, mask=mask)
    lines = lsd.detect(g)[0]
    if lines is None:
        return np.zeros((0,4), np.float32)
    segs = lines[:,0,:].astype(np.float32)
    L = np.hypot(segs[:,2]-segs[:,0], segs[:,3]-segs[:,1])
    segs = segs[L >= min_len]
    vis = cv2.cvtColor(g, cv2.COLOR_GRAY2BGR)
    for x1,y1,x2,y2 in segs.astype(int):
        cv2.line(vis, (x1,y1), (x2,y2), (0,255,0), 2, cv2.LINE_AA)
    return segs

def seg_angle(seg):
    x1,y1,x2,y2 = seg
    a = math.atan2(y2-y1, x2-x1)
    if a < 0: a += math.pi
    return a

def cluster_two_families(lines):
    if len(lines) < 2: return None, None
    ang = np.array([seg_angle(s) for s in lines], np.float32)
    feats = np.c_[np.cos(2*ang), np.sin(2*ang)].astype(np.float32)
    crit = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-3)
    _, lab, _ = cv2.kmeans(feats, 2, None, crit, 10, cv2.KMEANS_PP_CENTERS)
    lab = lab.ravel()
    return lines[lab==0], lines[lab==1]

def mean_angle_of_family(family):
    if len(family) == 0: return None
    ang = np.array([seg_angle(s) for s in family], np.float64)
    c = np.cos(2*ang).mean(); s = np.sin(2*ang).mean()
    a2 = math.atan2(s, c);  a2 = a2 if a2 >= 0 else a2 + 2*math.pi
    return 0.5 * a2

def angle_diff(a, b):
    d = abs(a - b);  return min(d, math.pi - d)

def draw_segments(img, segs, color=(255,0,0), thick=3):
    vis = img.copy()
    for x1,y1,x2,y2 in segs.astype(int):
        cv2.line(vis, (x1,y1), (x2,y2), color, thick, cv2.LINE_AA)
    return vis

# ---------- merging (preserve per-line angles) ----------
def _pca_dir_from_segments(segs):
    dirs=[]
    for s in segs:
        dx,dy = s[2]-s[0], s[3]-s[1]
        n = math.hypot(dx,dy) + 1e-12
        dirs.append([dx/n, dy/n])
    D = np.array(dirs, np.float64)
    if len(D) == 1: u = D[0]
    else:
        U,S,Vt = np.linalg.svd(D - D.mean(0))
        u = Vt[0]
    u /= (np.linalg.norm(u)+1e-12)
    return u

def _project_interval(seg, u, n):
    p1 = np.array([seg[0], seg[1]], dtype=np.float64)
    p2 = np.array([seg[2], seg[3]], dtype=np.float64)
    t1 = float(np.dot(u, p1)); t2 = float(np.dot(u, p2))
    t_lo, t_hi = (t1, t2) if t1 <= t2 else (t2, t1)
    mid = 0.5*(p1+p2)
    rho = float(np.dot(n, mid))
    return t_lo, t_hi, rho

def merge_family_preserve_angles(segs, theta_family, diag,
                                 rho_eps_ratio=RHO_EPS_RATIO, gap_eps_ratio=GAP_EPS_RATIO):
    if len(segs) == 0: return segs
    uF = np.array([math.cos(theta_family), math.sin(theta_family)], np.float64)
    nF = np.array([-uF[1], uF[0]], np.float64)
    rho_eps = rho_eps_ratio * diag
    gap_eps = gap_eps_ratio * diag
    pad = 0.02 * diag

    mids = 0.5*(segs[:,0:2] + segs[:,2:4])
    rhoF = mids @ nF
    order = np.argsort(rhoF)
    segs = segs[order]; rhoF = rhoF[order]

    merged = []
    i = 0; N = len(segs)
    while i < N:
        j = i+1
        while j < N and abs(rhoF[j]-rhoF[i]) <= rho_eps:
            j += 1
        bin_segs = segs[i:j]

        u = _pca_dir_from_segments(bin_segs)
        n = np.array([-u[1], u[0]], np.float64)

        items = []
        for s in bin_segs:
            t_lo, t_hi, rho = _project_interval(s, u, n)
            items.append([t_lo - pad, t_hi + pad, rho])
        items = np.array(items, np.float64)

        rho_mean = float(items[:,2].mean())
        intervals = items[:,0:2]
        intervals = intervals[np.argsort(intervals[:,0])]

        cur_lo, cur_hi = intervals[0,0], intervals[0,1]
        outs = []
        for k in range(1, len(intervals)):
            a,b = intervals[k]
            if a <= cur_hi + gap_eps: cur_hi = max(cur_hi, b)
            else: outs.append((cur_lo, cur_hi)); cur_lo, cur_hi = a, b
        outs.append((cur_lo, cur_hi))

        for (a,b) in outs:
            p1 = u*a + n*rho_mean
            p2 = u*b + n*rho_mean
            merged.append([p1[0], p1[1], p2[0], p2[1]])

        i = j

    return np.array(merged, np.float32)

def nms_by_rho(segs, theta, diag, eps_ratio=0.010):
    if len(segs)==0: return segs
    u = np.array([math.cos(theta), math.sin(theta)], np.float64)
    n = np.array([-u[1], u[0]], np.float64)
    rho = (0.5*(segs[:,0:2] + segs[:,2:4])) @ n
    order = np.argsort(rho)
    segs = segs[order]; rho = rho[order]
    eps = eps_ratio * diag
    kept = []; i = 0
    while i < len(segs):
        j = i+1
        best_k, best_len = i, np.hypot(*(segs[i,2:4]-segs[i,0:2]))
        while j < len(segs) and abs(rho[j]-rho[i]) <= eps:
            L = np.hypot(*(segs[j,2:4]-segs[j,0:2]))
            if L > best_len: best_k, best_len = j, L
            j += 1
        kept.append(segs[best_k]); i = j
    return np.array(kept, np.float32)

# ----- geometry helpers -----
def family_unit(theta):
    u = np.array([math.cos(theta), math.sin(theta)], np.float64)
    n = np.array([-u[1], u[0]], np.float64)
    return u, n

def edge_angle(a,b):
    return seg_angle(np.array([a[0],a[1],b[0],b[1]], np.float32))

def quad_edges(quad):
    return [(quad[i], quad[(i+1)%4]) for i in range(4)]

def family_board_rho_limits(quad, theta):
    u, n = family_unit(theta)
    edges = quad_edges(quad)
    angs = [edge_angle(a,b) for a,b in edges]
    diffs = [angle_diff(ang, theta) for ang in angs]
    idx = np.argsort(diffs)[:2]
    rhos = [np.dot(n, 0.5*(edges[i][0]+edges[i][1])) for i in idx]
    rho_min, rho_max = (min(rhos), max(rhos))
    all_rho = [np.dot(n, p) for p in quad]
    span = max(all_rho) - min(all_rho)
    if (rho_max - rho_min) < 0.3*span:
        rho_min, rho_max = min(all_rho), max(all_rho)
    return rho_min, rho_max

# ----- texture / measures -----
def bilinear_sample(gray, x, y):
    h,w = gray.shape
    if x<0 or y<0 or x>w-1 or y>h-1: return None
    x0,y0 = int(np.floor(x)), int(np.floor(y))
    x1,y1 = min(x0+1,w-1), min(y0+1,h-1)
    a,b = x-x0, y-y0
    v = (1-a)*(1-b)*gray[y0,x0] + a*(1-b)*gray[y0,x1] + (1-a)*b*gray[y1,x0] + a*b*gray[y1,x1]
    return float(v)

def cross_section_score(gray, seg, n_samples=XSEC_NSAMPLES, half=XSEC_HALF_PIX):
    x1,y1,x2,y2 = seg
    vx, vy = x2-x1, y2-y1
    L = math.hypot(vx, vy) + 1e-9
    ux, uy = vx/L, vy/L
    nx, ny = -uy, ux
    scores = []
    for t in np.linspace(0.15, 0.85, n_samples):
        cx, cy = x1 + t*vx, y1 + t*vy
        vals = []
        for s in range(-half, half+1):
            v = bilinear_sample(gray, cx + s*nx, cy + s*ny)
            if v is not None: vals.append(v)
        if len(vals) >= 5:
            scores.append(np.std(vals))
    return float(np.mean(scores)) if scores else 0.0

# ----- line algebra & drawing -----
def seg_to_line(seg):
    x1,y1,x2,y2 = seg
    p1 = np.array([x1,y1,1.0]); p2 = np.array([x2,y2,1.0])
    l = np.cross(p1, p2)
    n = np.linalg.norm(l[:2]) + 1e-12
    return l / n

def safe_endpoints_from_line_in_image(l, w, h):
    a, b, c = float(l[0]), float(l[1]), float(l[2])
    eps = 1e-12
    n2 = a*a + b*b
    if n2 < eps:
        return (0.0, 0.0), (w-1.0, h-1.0)
    p0x = (-c) * a / n2
    p0y = (-c) * b / n2
    un = math.hypot(-b, a)
    ux, uy = (-b / max(un,1e-12), a / max(un,1e-12))

    x_min, x_max = -1.0, w + 1.0
    y_min, y_max = -1.0, h + 1.0
    cand = []
    if abs(ux) > eps:
        for X in (x_min, x_max-1.0):
            t = (X - p0x) / ux; y = p0y + t*uy
            if y_min <= y <= y_max-1.0: cand.append((X, y))
    if abs(uy) > eps:
        for Y in (y_min, y_max-1.0):
            t = (Y - p0y) / uy; x = p0x + t*ux
            if x_min <= x <= x_max-1.0: cand.append((x, Y))
    uniq = []
    for x,y in cand:
        if not any((abs(x-x2)<1e-6 and abs(y-y2)<1e-6) for (x2,y2) in uniq):
            uniq.append((x,y))
    if len(uniq) >= 2:
        best, bestd = (uniq[0], uniq[1]), -1.0
        for i in range(len(uniq)):
            for j in range(i+1,len(uniq)):
                dx = uniq[i][0]-uniq[j][0]; dy = uniq[i][1]-uniq[j][1]
                d2 = dx*dx + dy*dy
                if d2 > bestd: bestd, best = d2, (uniq[i], uniq[j])
        def clamp(p): return (max(0.0,min(w-1.0,p[0])), max(0.0,min(h-1.0,p[1])))
        return clamp(best[0]), clamp(best[1])

    t_big = max(w,h)*2.0
    p1x, p1y = p0x - t_big*ux, p0y - t_big*uy
    p2x, p2y = p0x + t_big*ux, p0y + t_big*uy
    p1x = max(0.0, min(w-1.0, p1x)); p1y = max(0.0, min(h-1.0, p1y))
    p2x = max(0.0, min(w-1.0, p2x)); p2y = max(0.0, min(h-1.0, p2y))
    return (p1x, p1y), (p2x, p2y)

def draw_full_lines(img, segs, color, thick=2):
    vis = img.copy()
    h, w = img.shape[:2]
    for s in segs:
        l = seg_to_line(s)
        (x1,y1),(x2,y2) = safe_endpoints_from_line_in_image(l, w, h)
        if (abs(x1-x2)+abs(y1-y2)) < 1.0:
            a,b,c = float(l[0]), float(l[1]), float(l[2])
            n2 = a*a + b*b
            if n2 < 1e-12: continue
            p0x = (-c) * a / n2; p0y = (-c) * b / n2
            un = math.hypot(-b, a); ux, uy = (-b/max(un,1e-12), a/max(un,1e-12))
            R = 100.0
            x1,y1 = p0x-R*ux, p0y-R*uy
            x2,y2 = p0x+R*ux, p0y+R*uy
            x1 = max(0.0, min(w-1.0, x1)); y1 = max(0.0, min(h-1.0, y1))
            x2 = max(0.0, min(w-1.0, x2)); y2 = max(0.0, min(h-1.0, y2))
        cv2.line(vis, (int(round(x1)),int(round(y1))), (int(round(x2)),int(round(y2))), color, thick, cv2.LINE_AA)
    return vis

def all_intersections(fam1_lines, fam2_lines, shape, snap_margin_frac=0.15):
    h, w = shape[:2]
    L1 = [seg_to_line(s) for s in fam1_lines]
    L2 = [seg_to_line(s) for s in fam2_lines]

    # how far outside we still allow snapping (e.g. 15% of size)
    max_dx = snap_margin_frac * w
    max_dy = snap_margin_frac * h

    pts = []
    for l1 in L1:
        for l2 in L2:
            p = np.cross(l1, l2)
            if abs(p[2]) < 1e-12:
                continue

            x = float(p[0] / p[2])
            y = float(p[1] / p[2])

            # if clearly somewhere crazy, ignore it
            if not np.isfinite(x) or not np.isfinite(y):
                continue

            # inside the image (with small margin) -> keep as is
            if -2 <= x <= w + 2 and -2 <= y <= h + 2:
                pts.append([x, y])
                continue

            # slightly outside -> snap to border
            dx = 0
            if x < 0:      dx = -x
            elif x > w:    dx = x - w
            dy = 0
            if y < 0:      dy = -y
            elif y > h:    dy = y - h

            if dx <= max_dx and dy <= max_dy:
                x_clipped = min(max(x, 0.0), w - 1.0)
                y_clipped = min(max(y, 0.0), h - 1.0)
                pts.append([x_clipped, y_clipped])
                continue

            # too far away -> drop

    return np.array(pts, np.float32) if pts else np.zeros((0, 2), np.float32)
def all_intersections(fam1_lines, fam2_lines, shape, snap_margin_frac=0.15):
    h, w = shape[:2]
    L1 = [seg_to_line(s) for s in fam1_lines]
    L2 = [seg_to_line(s) for s in fam2_lines]

    # how far outside we still allow snapping (e.g. 15% of size)
    max_dx = snap_margin_frac * w
    max_dy = snap_margin_frac * h

    pts = []
    for l1 in L1:
        for l2 in L2:
            p = np.cross(l1, l2)
            if abs(p[2]) < 1e-12:
                continue

            x = float(p[0] / p[2])
            y = float(p[1] / p[2])

            # if clearly somewhere crazy, ignore it
            if not np.isfinite(x) or not np.isfinite(y):
                continue

            # inside the image (with small margin) -> keep as is
            if -2 <= x <= w + 2 and -2 <= y <= h + 2:
                pts.append([x, y])
                continue

            # slightly outside -> snap to border
            dx = 0
            if x < 0:      dx = -x
            elif x > w:    dx = x - w
            dy = 0
            if y < 0:      dy = -y
            elif y > h:    dy = y - h

            if dx <= max_dx and dy <= max_dy:
                x_clipped = min(max(x, 0.0), w - 1.0)
                y_clipped = min(max(y, 0.0), h - 1.0)
                pts.append([x_clipped, y_clipped])
                continue

            # too far away -> drop

    return np.array(pts, np.float32) if pts else np.zeros((0, 2), np.float32)


# ---------- coverage & span utilities ----------
def line_board_coverage(seg, mask):
    if mask is None: return 1.0
    h, w = mask.shape[:2]
    x1,y1,x2,y2 = map(float, seg)
    L = max(1.0, math.hypot(x2-x1, y2-y1))
    nS = int(max(20, L/8))
    inside = 0
    for t in np.linspace(0.05, 0.95, nS):
        x = x1 + t*(x2-x1); y = y1 + t*(y2-y1)
        xi = int(round(min(max(0, x), w-1)))
        yi = int(round(min(max(0, y), h-1)))
        inside += 1 if mask[yi, xi] > 0 else 0
    return inside / nS

def rho_span_and_d(theta, quad, n):
    uF, nF = family_unit(theta)
    rmin, rmax = family_board_rho_limits(quad, theta)
    d = (rmax - rmin) / max(n-1, 1)
    return rmin, rmax, d, nF

def filter_pool_by_inner_band(pool, theta, quad, n, margin_ratio, nF, mask_inner):
    """Keep only candidates whose ρ is within the inner band AND have good coverage."""
    if len(pool) == 0:
        return pool, 0.0
    rmin, rmax = family_board_rho_limits(quad, theta)
    d = (rmax - rmin) / max(n-1, 1)
    lo = rmin + margin_ratio * d
    hi = rmax - margin_ratio * d

    mids = 0.5*(pool[:,0:2] + pool[:,2:4])
    rho  = (mids @ nF).astype(np.float64)
    cov  = np.array([line_board_coverage(s, mask_inner) for s in pool], np.float64)

    keep = (rho >= lo) & (rho <= hi) & (cov >= MIN_COVER)
    return pool[keep], d

def select_n_hungarian_from_pool(pool, theta, n, gray, quad, mask_inner):
    """
    Pick ~n lines from 'pool' by assigning them to n evenly-spaced centers
    inside an inner band; fall back by relaxing MIN_SEP_MULT if we get < n.
    """
    uF, nF = family_unit(theta)

    # board span
    rmin, rmax = family_board_rho_limits(quad, theta)
    d = (rmax - rmin) / max(n-1, 1)

    # Use inner-band centers (prevents picking extreme rim)
    lo = rmin + INNER_BAND_MARGIN * d
    hi = rmax - INNER_BAND_MARGIN * d
    d_in = (hi - lo) / max(n-1, 1)
    centers = lo + d_in * np.arange(n, dtype=np.float64)

    # measures
    if len(pool) == 0:
        return pool

    mids = 0.5*(pool[:,0:2] + pool[:,2:4])
    rho  = (mids @ nF).astype(np.float64)
    L    = np.hypot(pool[:,2]-pool[:,0], pool[:,3]-pool[:,1]).astype(np.float64)
    Adev = np.array([angle_diff(seg_angle(s), theta) for s in pool], np.float64)
    tex  = np.array([cross_section_score(gray, s, n_samples=max(8, XSEC_NSAMPLES//2), half=XSEC_HALF_PIX)
                     for s in pool], np.float64)
    cov  = np.array([line_board_coverage(s, mask_inner) for s in pool], np.float64)

    Lq0,Lq1 = np.quantile(L,0.10), np.quantile(L,0.90)
    len_norm = np.clip((L - Lq0) / (Lq1 - Lq0 + 1e-9), 0, 1)
    ang_norm = max(math.radians(ANG_NORM_DEG), 1e-3)
    dang_norm = np.clip(Adev/ang_norm, 0, 3.0)
    tq0,tq1 = np.quantile(tex,0.20), np.quantile(tex,0.90)
    tex_norm = np.clip((tex - tq0) / (tq1 - tq0 + 1e-9), 0, 1)
    Q = (1.0 - BETA_ANG*dang_norm) * (BETA_LEN0 + BETA_LEN1*len_norm) * (0.2 + TEXT_BONUS*tex_norm)

    # base cost
    C_base = np.zeros((len(pool), n), dtype=np.float64)
    for j, c in enumerate(centers):
        C_base[:, j] = W_RHO * np.abs(rho - c) + W_Q * (1.0 - Q)

    # hard gates (do once, reused for all separation levels)
    C = C_base.copy()
    far_gate = FAR_GATE_MULT * d_in
    out_of_band = (rho < lo - 0.25*d) | (rho > hi + 0.25*d)
    too_far     = np.min(np.abs(rho[:,None] - centers[None,:]), axis=1) > far_gate
    bad_cover   = cov < MIN_COVER
    kill = out_of_band | too_far | bad_cover
    if np.any(kill):
        C[kill, :] += 1e6  # make them unselectable

    def run_with_min_sep(min_sep_mult: float):
        """Run assignment + separation + refill with a given separation multiplier."""
        if len(pool) == 0:
            return pool

        # assignment
        rows, cols = linear_sum_assignment(C)
        sel_rows = rows[(cols >= 0) & (cols < n)]
        chosen = pool[sel_rows]

        # order by rho
        mids_ch = 0.5*(chosen[:,0:2] + chosen[:,2:4])
        rho_ch  = mids_ch @ nF
        order   = np.argsort(rho_ch)
        chosen  = chosen[order]
        rho_ch  = rho_ch[order]

        # separation
        min_sep = min_sep_mult * d_in
        ok = [True]*len(chosen)
        for i in range(1, len(chosen)):
            if abs(rho_ch[i] - rho_ch[i-1]) < min_sep:
                qi = Q[sel_rows[order[i]]]   * max(cov[sel_rows[order[i]]],   1e-9)
                qj = Q[sel_rows[order[i-1]]] * max(cov[sel_rows[order[i-1]]], 1e-9)
                ok[i-1 if qi >= qj else i] = False
        chosen = chosen[np.array(ok, bool)]
        used = set(sel_rows[order][np.array(ok, bool)])

        # refill if we dropped any
        if len(chosen) < n:
            best_center_cost = C.min(axis=1)
            for idx in np.argsort(best_center_cost):
                if idx in used or best_center_cost[idx] > 1e5:
                    continue
                s = pool[idx]
                r = float((0.5*(s[0:2]+s[2:4]) @ nF))
                if (r < lo) or (r > hi) or (cov[idx] < MIN_COVER):
                    continue
                if len(chosen) > 0 and min_sep > 0:
                    rho_cur = (0.5*(chosen[:,0:2]+chosen[:,2:4]) @ nF)
                    if np.min(np.abs(rho_cur - r)) < min_sep:
                        continue
                chosen = np.vstack([chosen, s]) if len(chosen) else np.array([s], np.float32)
                used.add(idx)
                if len(chosen) == n:
                    break

        # final in-band clamp
        if len(chosen) == 0:
            return chosen

        mids_fin = 0.5*(chosen[:,0:2] + chosen[:,2:4])
        rho_fin  = mids_fin @ nF
        keep = (rho_fin >= lo) & (rho_fin <= hi)
        chosen = chosen[keep]
        if len(chosen) == 0:
            return chosen

        # sort by rho again
        mids_fin = 0.5*(chosen[:,0:2] + chosen[:,2:4])
        rho_fin  = mids_fin @ nF
        chosen   = chosen[np.argsort(rho_fin)]
        return chosen

    # --- try with decreasing separation ---
    chosen_final = np.empty((0,4), dtype=np.float32)

    # e.g. MIN_SEP_MULT, MIN_SEP_MULT-0.1, ..., down to 0
    for step in range(8):
        cur_mult = max(MIN_SEP_MULT - 0.1*step, 0.0)
        chosen = run_with_min_sep(cur_mult)
        if len(chosen) == 0:
            continue
        if len(chosen) >= n:
            chosen_final = chosen
            break
        # keep best so far if we haven't reached n yet
        if len(chosen) > len(chosen_final):
            chosen_final = chosen

    if len(chosen_final) == 0:
        # totally failed; just return all pool sorted by rho
        mids_all = 0.5*(pool[:,0:2] + pool[:,2:4])
        rho_all  = mids_all @ nF
        pool_sorted = pool[np.argsort(rho_all)]
        # if still more than n, sub-sample
        if len(pool_sorted) > n:
            idx = np.linspace(0, len(pool_sorted)-1, n).astype(int)
            pool_sorted = pool_sorted[idx]
        return pool_sorted.astype(np.float32)

    # if we ended up with more than n, thin them out
    if len(chosen_final) > n:
        idx = np.linspace(0, len(chosen_final)-1, n).astype(int)
        chosen_final = chosen_final[idx]

    return chosen_final.astype(np.float32)


In [ ]:
def getGreenMask(rgb_image: np.ndarray):
    """
    Detect green regions (RGB input).
    Returns:
        green_mask: 0/255
        green_detected: RGB image with only green regions
    """
    hsv = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)

    lower_green = np.array([40, 20, 30], dtype=np.uint8)
    upper_green = np.array([100, 255, 200], dtype=np.uint8)

    green_mask = cv2.inRange(hsv, lower_green, upper_green)

    kernel = np.ones((20, 20), np.uint8)
    green_mask_ = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    green_mask  = cv2.erode(green_mask_, kernel, iterations=1)

    green_detected = cv2.bitwise_and(rgb_image, rgb_image, mask=green_mask)
    return green_mask, green_detected

In [ ]:
def GetCorner(binary_img: np.ndarray):
    """
    Return (y_min, x_min, y_max, x_max) or None.
    """
    y_idx, x_idx = np.where(binary_img == 255)
    if len(x_idx) == 0:
        return None

    y_min, y_max = y_idx.min(), y_idx.max()
    x_min, x_max = x_idx.min(), x_idx.max()
    return y_min, x_min, y_max, x_max

In [ ]:
def make_two_crops_from_bbox(frame_bgr: np.ndarray,
                             bbox,
                             pad_grid: int,
                             pad_img: int):
    """
    Given original frame and bounding box (y_min, x_min, y_max, x_max),
    make:
      - board_for_grid_bgr : crop with pad_grid
      - raw_board_bgr      : crop with pad_img (usually larger)

    Also return the (x, y) top-left origins of each crop in the ORIGINAL frame.
    """
    y_min, x_min, y_max, x_max = bbox
    H, W = frame_bgr.shape[:2]

    # ---- crop for grid (smaller padding) ----
    yg1 = max(0, y_min - pad_grid)
    yg2 = min(H, y_max + pad_grid)
    xg1 = max(0, x_min - pad_grid)
    xg2 = min(W, x_max + pad_grid)
    board_for_grid = frame_bgr[yg1:yg2, xg1:xg2, :]

    # ---- crop for raw image (bigger padding) ----
    yi1 = max(0, y_min - pad_img)
    yi2 = min(H, y_max + pad_img)
    xi1 = max(0, x_min - pad_img)
    xi2 = min(W, x_max + pad_img)
    raw_board = frame_bgr[yi1:yi2, xi1:xi2, :]

    # return images + their top-left coords in the original frame
    grid_origin = (xg1, yg1)   # (x, y) of board_for_grid[0,0] in original frame
    img_origin  = (xi1, yi1)   # (x, y) of raw_board[0,0]    in original frame

    return board_for_grid, raw_board, grid_origin, img_origin

In [ ]:
def get_roi(
    frame_bgr: np.ndarray,
    grid_padding: int = 40,
    image_padding: int = 120,
):
    grid_offset_vec = np.zeros(2, dtype=np.float32)
    board_img = None
    chess_img = None
    # 1. หาพื้นที่สีเขียว (Green Mask) เพื่อระบุตำแหน่งกระดาน
    green_mask, _ = getGreenMask(cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB))
    bbox = GetCorner(green_mask) # หาจุดมุม 4 จุด
        
    if bbox is not None:
        board_img, chess_img, grid_origin, img_origin = make_two_crops_from_bbox(frame_bgr, bbox,pad_grid=grid_padding, pad_img=image_padding,) 
        gx1, gy1 = grid_origin   # top-left of board_img in original frame
        ix1, iy1 = img_origin    # top-left of chess_img in original frame

        grid_offset_vec = np.array([gx1 - ix1, gy1 - iy1], dtype=np.float32)
            
    return board_img, chess_img, grid_offset_vec

## Hand Detection

In [ ]:
def hand_detection(frame, cut_bound, skin_threshold = 0.00056, debug = False):
    
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    
    # Skin color range (tune if necessary)
    lower_skin = np.array([0, 135, 85], dtype=np.uint8)
    upper_skin = np.array([255, 180, 135], dtype=np.uint8)

    mask = cv2.inRange(ycrcb, lower_skin, upper_skin)
    
    # Convert to 0/1
    mask_binary = (mask > 0).astype(np.uint8)
    
    h, w = mask_binary.shape
    
    # ตรวจสอบว่า cut_bound มีค่าและไม่มากเกินขนาดภาพ
    if cut_bound > 0:
        mask_binary[:cut_bound, :] = 0        # ขอบบน
        mask_binary[-cut_bound:, :] = 0       # ขอบล่าง
        mask_binary[:, :cut_bound] = 0        # ขอบซ้าย
        mask_binary[:, -cut_bound:] = 0
        
    mask_binary = cv2.morphologyEx(mask_binary, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=3)
    mask_binary = cv2.dilate(mask_binary, np.ones((5,5), np.uint8), iterations=3)
    
    skin_pixels = np.sum(mask_binary)
    total_pixels = mask_binary.size 
    skin_ratio = skin_pixels/total_pixels
    have_hand = skin_ratio > skin_threshold
    
    if debug:
        plt.figure(figsize=(10, 4))
        
        # รูปซ้าย: ต้นฉบับ
        plt.subplot(1, 2, 1)
        # วาดกรอบสีเขียวเพื่อให้รู้ว่าเราตัดตรงไหนมาคิดคะแนน
        img_show = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if cut_bound > 0:
            cv2.rectangle(img_show, (cut_bound, cut_bound), (w-cut_bound, h-cut_bound), (0, 255, 0), 2)
        plt.imshow(img_show)
        plt.title(f"Original (Skin Ratio: {skin_ratio})")
        plt.axis("off")
        
        # รูปขวา: ควรโชว์ roi_mask ที่ใช้คำนวณจริง
        plt.subplot(1, 2, 2)
        plt.imshow(mask_binary, cmap='gray') # แก้จาก mask เป็น roi_mask
        plt.title("Skin Mask (Processed)")
        plt.axis("off")
        
        plt.show()
    
    return have_hand

## Deduplication By Pixel Similarity  Check

In [ ]:
def dedup_by_pixel_similarity_check(current_img, prev_img, cut_bound, threshold=0.0033, debug=False):
    if prev_img is None or current_img is None:
        return False, None

    # 1. Resize ให้เท่ากัน
    h, w = current_img.shape[:2]
    
    prev_resized = cv2.resize(prev_img, (w, h))
    
    # 2. Pre-process (Gray + Blur)
    curr_gray = cv2.cvtColor(current_img, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.cvtColor(prev_resized, cv2.COLOR_BGR2GRAY)

    curr_blur = cv2.GaussianBlur(curr_gray, (5, 5), 0)
    prev_blur = cv2.GaussianBlur(prev_gray, (5, 5), 0)

    # 3. AbsDiff & Threshold
    diff = cv2.absdiff(curr_blur, prev_blur)
    _, binary_mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # ---------------------------------------------------------
    # 4. ***Morphological : clear noise***
    kernel = np.ones((3, 3), np.uint8)
    
    # ลบเส้นบางๆ ทิ้ง
    cleaned_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    cleaned_mask = cv2.dilate(cleaned_mask, kernel, iterations=3)
    # ---------------------------------------------------------
    # ตัดส่วนที่เป็นนอกจาก board ทิ้ง
    if cut_bound > 0:
        cleaned_mask[:cut_bound, :] = 0
        cleaned_mask[-cut_bound:, :] = 0
        cleaned_mask[:, :cut_bound] = 0
        cleaned_mask[:, -cut_bound:] = 0
        
    # 5. Count Non-Zero จาก "Cleaned Mask"
    changed_pixels = cv2.countNonZero(cleaned_mask)
    total_pixels = cleaned_mask.size
    changed_ratio = changed_pixels/total_pixels
    is_dup = changed_ratio < threshold
    
    # --- Debug Logic ---
    if debug:
        print(f"-"*50)
        print(f"Changed Pixels Ratio: {changed_ratio} (Threshold: {threshold})")
        print(f"Result: {'DUPLICATE' if is_dup else 'NEW MOVE'}")
        
        fig, axes = plt.subplots(1, 4, figsize=(20, 5)) # เพิ่มช่องโชว์ Cleaned Mask
        
        # ภาพดิบ
        axes[0].imshow(cv2.cvtColor(prev_resized, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Previous")
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(current_img, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f"Current")
        axes[1].axis('off')

        # Mask ดิบ (ยังมีเส้นรบกวน)
        axes[2].imshow(binary_mask, cmap='gray') 
        axes[2].set_title("Raw Mask (Noise included)")
        axes[2].axis('off')

        # Mask ที่คลีนแล้ว (ใช้คำนวณจริง)
        axes[3].imshow(cleaned_mask, cmap='gray') 
        axes[3].set_title(f"Cleaned Mask\nPixel Diff: {changed_pixels}")
        axes[3].axis('off')
        
        plt.tight_layout()
        plt.show()

    return is_dup, cleaned_mask

## Chess Piece Detection

In [ ]:
def get_predictions_json(
    img_path: str,
    model: Any, 
    confidence: int = 40, 
    overlap: int = 60
) -> Dict[str, Any]:

    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Error: Image file not found at path: {img_path}")
        
    # 1. Convert parameters (Roboflow uses 0-100, YOLO uses 0.0-1.0)
    conf_threshold = confidence / 100.0
    iou_threshold = overlap / 100.0

    # 2. Run Inference
    # verbose=False prevents it from printing details to console
    results = model.predict(
        source=img_path, 
        conf=conf_threshold, 
        iou=iou_threshold, 
        agnostic_nms=True,
        verbose=False
    )
    
    # 3. Format results to mimic Roboflow JSON structure
    predictions = []
    
    # We only processed one image, so we take results[0]
    result = results[0]
    
    for box in result.boxes:
        # Extract data from the tensor
        # xywh returns center_x, center_y, width, height (same as Roboflow)
        x, y, w, h = box.xywh[0].tolist()
        conf = box.conf[0].item()
        cls_id = int(box.cls[0].item())
        class_name = result.names[cls_id]
        
        # Build the dictionary entry
        pred = {
            "x": x,
            "y": y,
            "width": w,
            "height": h,
            "class": class_name,
            "class_id": cls_id,     # Helpful for your PIECE_MAP
            "confidence": conf
        }
        predictions.append(pred)
    
    # Return dictionary with 'predictions' key
    return {"predictions": predictions}

## Board Matrix Construction

In [ ]:
CLASS_TO_FEN = {
    "white-pawn":   "P",
    "white-knight": "N",
    "white-bishop": "B",
    "white-rook":   "R",
    "white-queen":  "Q",
    "white-king":   "K",
    "black-pawn":   "p",
    "black-knight": "n",
    "black-bishop": "b",
    "black-rook":   "r",
    "black-queen":  "q",
    "black-king":   "k",
}

In [ ]:
def detect_grid(board_img, out_dir):
    """
    Run LSD + clustering + Hungarian grid detection on a *cropped board image*.
    NO border contour detection; the whole image is treated as the board quad.

    Returns:
        img_out : image in the SAME coordinate system as board_img
        fam1_s  : vertical line segments in board_img coords
        fam2_s  : horizontal line segments in board_img coords
        dots_s  : intersections (N,2) in board_img coords
    """
    os.makedirs(out_dir, exist_ok=True)

    # --- optionally resize for speed ---
    h0, w0 = board_img.shape[:2]
    if w0 > MAX_W:
        s = MAX_W / w0
        img = cv2.resize(board_img, (int(w0 * s), int(h0 * s)), cv2.INTER_AREA)
    else:
        img = board_img.copy()
    h, w = img.shape[:2]
    diag = math.hypot(w, h)

    # 1) edges
    edges = preprocess_edges(img)
    cv2.imwrite(str(out_dir / "01_edges.jpg"), edges)

    # 2) full-image quad
    quad = np.array([
        [0,     0    ],
        [w - 1, 0    ],
        [w - 1, h - 1],
        [0,     h - 1]
    ], dtype=np.float32)

    mask_board = np.full((h, w), 255, np.uint8)
    mask_inner = erode_mask_by_ratio(mask_board, MASK_ERODE_RATIO_INNER)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3) LSD segments
    all_segs  = lsd_segments(gray, mask=mask_inner, min_len_ratio=0.0)
    long_segs = lsd_segments(gray, mask=mask_inner, min_len_ratio=MIN_LEN_RATIO)
    assert len(long_segs) >= 2, "need long segments for clustering"

    # 4) cluster long lines into two angle families
    fam1_long, fam2_long = cluster_two_families(long_segs)
    theta1 = mean_angle_of_family(fam1_long)
    theta2 = mean_angle_of_family(fam2_long)

    # 5) filter ALL segments by angle proximity
    ang_tol = math.radians(ANGLE_TOL_DEG)
    cand1, cand2 = [], []
    for s in all_segs:
        a = seg_angle(s)
        if angle_diff(a, theta1) <= ang_tol:
            cand1.append(s)
        elif angle_diff(a, theta2) <= ang_tol:
            cand2.append(s)
    cand1 = np.array(cand1, np.float32)
    cand2 = np.array(cand2, np.float32)

    # 6) merge + NMS + length filter
    fam1_merged = merge_family_preserve_angles(cand1, theta1, diag, RHO_EPS_RATIO, GAP_EPS_RATIO)
    fam2_merged = merge_family_preserve_angles(cand2, theta2, diag, RHO_EPS_RATIO, GAP_EPS_RATIO)
    fam1_merged = nms_by_rho(fam1_merged, theta1, diag, eps_ratio=0.010)
    fam2_merged = nms_by_rho(fam2_merged, theta2, diag, eps_ratio=0.010)

    def _filter_by_len(segs, min_len):
        if len(segs) == 0:
            return segs
        L = np.hypot(segs[:,2]-segs[:,0], segs[:,3]-segs[:,1])
        return segs[L >= min_len]

    min_len_after = POST_MIN_LEN_RATIO * diag
    fam1_before = _filter_by_len(fam1_merged, min_len_after)
    fam2_before = _filter_by_len(fam2_merged, min_len_after)

    # 7) drop rim-ish lines using inner band and coverage
    rmin1, rmax1, d1, nF1 = rho_span_and_d(theta1, quad, N_GRID)
    rmin2, rmax2, d2, nF2 = rho_span_and_d(theta2, quad, N_GRID)
    fam1_pool, d1 = filter_pool_by_inner_band(
        fam1_before, theta1, quad, N_GRID, INNER_BAND_MARGIN, nF1, mask_inner
    )
    fam2_pool, d2 = filter_pool_by_inner_band(
        fam2_before, theta2, quad, N_GRID, INNER_BAND_MARGIN, nF2, mask_inner
    )

    # debug: merged before select
    vis_before = draw_full_lines(img, fam1_pool, (255, 0, 0), 3)
    vis_before = draw_full_lines(vis_before, fam2_pool, (0, 255, 0), 3)
    cv2.imwrite(str(out_dir / "03b_merged_before_select.jpg"), vis_before)

    # 8) strict selection of exactly N_GRID per family
    fam1 = select_n_hungarian_from_pool(fam1_pool, theta1, N_GRID, gray, quad, mask_inner)
    fam2 = select_n_hungarian_from_pool(fam2_pool, theta2, N_GRID, gray, quad, mask_inner)

    # 9) intersections in the RESIZED image coords
    dots_resized = all_intersections(fam1, fam2, img.shape)

    # Debug images (still in resized coords)
    vis_selected = draw_full_lines(img, fam1, (255, 0, 0), 3)
    vis_selected = draw_full_lines(vis_selected, fam2, (0, 255, 0), 3)
    cv2.imwrite(str(out_dir / "03c_merged_selected_NxN.jpg"), vis_selected)

    orig_vis = vis_selected.copy()
    for (x, y) in dots_resized.astype(int):
        cv2.circle(orig_vis, (x, y), 3, (0, 0, 255), -1)
    cv2.imwrite(str(out_dir / "04_output_on_original_NxN.jpg"), orig_vis)

    print("Saved debug to:", out_dir)
    print(f"Selected - Fam1: {len(fam1)}, Fam2: {len(fam2)}, Intersections: {len(dots_resized)}")

    # ---------- SCALE BACK to original board_img coords ----------
    if w0 > MAX_W:
        sx = w0 / float(w)
        sy = h0 / float(h)

        dots_s = dots_resized.copy()
        dots_s[:, 0] *= sx
        dots_s[:, 1] *= sy

        fam1_s = fam1.copy()
        fam2_s = fam2.copy()
        fam1_s[:, [0, 2]] *= sx
        fam1_s[:, [1, 3]] *= sy
        fam2_s[:, [0, 2]] *= sx
        fam2_s[:, [1, 3]] *= sy

        img_out = board_img  # external users expect coords in board_img space
    else:
        img_out = img
        fam1_s, fam2_s = fam1, fam2
        dots_s = dots_resized

    return img_out, fam1_s, fam2_s, dots_s



In [ ]:
def dots_to_grid(dots, n_lines=9):
    """
    Convert unordered intersections into a [n_lines, n_lines, 2] grid of points
    sorted by image coordinates (row=top→bottom, col=left→right).
    dots: (N,2) float32 array from all_intersections
    """
    assert dots.shape[1] == 2
    ys = dots[:, 1].reshape(-1, 1)

    # Cluster by Y into n_lines horizontal groups
    kmeans = KMeans(n_clusters=n_lines, n_init=10, random_state=0)
    labels = kmeans.fit_predict(ys)
    centers = kmeans.cluster_centers_.ravel()

    # Map cluster index -> row index (0 = top)
    cluster_order = np.argsort(centers)
    cluster_to_row = {c: i for i, c in enumerate(cluster_order)}

    grid = np.zeros((n_lines, n_lines, 2), dtype=np.float32)

    for c in range(n_lines):
        # points in this cluster (row)
        pts = dots[labels == c]
        row_idx = cluster_to_row[c]

        # sort this row by x coordinate
        order_x = np.argsort(pts[:, 0])
        pts_sorted = pts[order_x]

        # if more than n_lines points (rare), keep closest n_lines
        if len(pts_sorted) > n_lines:
            pts_sorted = pts_sorted[:n_lines]

        # if fewer, pad by interpolation (very rare)
        if len(pts_sorted) < n_lines:
            # simple linear interpolation on x
            xs = np.linspace(pts_sorted[0,0], pts_sorted[-1,0], n_lines)
            ys_row = np.interp(xs, pts_sorted[:,0], pts_sorted[:,1])
            pts_sorted = np.stack([xs, ys_row], axis=1).astype(np.float32)

        grid[row_idx, :, :] = pts_sorted

    return grid  # shape (9,9,2)


In [ ]:
def build_board_from_predictions(pred_json, grid, min_coverage_frac: float = 0.10):
    # --- 1. เตรียมข้อมูล Grid และพื้นที่กระดานรวม (Entire Grid Box) ---
    # grid shape: (9, 9, 2)
    row_lines_y = np.sort(grid.mean(axis=1)[:, 1])
    col_lines_x = np.sort(grid.mean(axis=0)[:, 0])
    
    # ขอบเขตของกระดานทั้งหมด (เพื่อใช้ตัดส่วนที่หลุดออกไป)
    board_min_x = col_lines_x[0]
    board_max_x = col_lines_x[-1]
    board_min_y = row_lines_y[0]
    board_max_y = row_lines_y[-1]
    entire_board_box = (board_min_x, board_min_y, board_max_x, board_max_y)

    # สร้างกล่อง 64 ช่องเตรียมไว้
    square_boxes = [[None]*8 for _ in range(8)]
    for r in range(8):
        for c in range(8):
            square_boxes[r][c] = (
                col_lines_x[c],     # x1
                row_lines_y[r],     # y1
                col_lines_x[c+1],   # x2
                row_lines_y[r+1]    # y2
            )
            
    """
    chess -> ex. K = (<0,7> 0.5), (<1,7> 0.5)
    loop -> ใส่ อัตราส่วน+chess piece ex. 0.5K ใช้ dict
    อัตราส่วน = ดูว่าตัวหมาก "ล้ำ" เข้าไปในช่องนั้นกี่เปอร์เซ็นต์ (Intersection Area / Piece Area)
    ปล. ถ้า chess piece นั้นมีส่วนที่หลุดออกไปนอก grid (ไม่อยู่ใน grid ไหนเลย) piece area ต้องใช้เฉพาะที่อยู่ใน grid
    dict = {<coordinate> : [chesspiece, percentage]...}
    ex. {<0,7> : [(0.5,K), (0.4,B)]} = ช่อง <row = 0, col = 7> มีผู้ท้าชิงสองคน
    เริ่มจากช่องที่มีผู้ท้าชิงคนเดียวก่อน อย่าลืมตัดเรื่อยๆ ว่ามีหมากอะไรใช้ไปแล้วบ้าง ตอนทำต่อๆ ไปก็อย่าใช้หมากที่ไม่มีให้ใช้แล้ว
    """
    
    # Helper function: คำนวณพื้นที่ Intersection
    def get_inter_area(boxA, boxB):
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        return max(0, xB - xA) * max(0, yB - yA)

    # --- 2. สร้าง Dictionary Candidates { (r,c): [list of pieces] } ---
    # key: (r, c), value: list of {'id': int, 'fen': str, 'score': float}
    square_candidates = defaultdict(list)
    
    # Map Class -> FEN
    cls_to_fen = CLASS_TO_FEN # (ใช้ตัวแปร Global ที่คุณมีอยู่แล้ว)

    # วนลูปดูตัวหมากทุกตัวที่ AI เจอ
    predictions = pred_json.get("predictions", [])
    valid_pieces_count = 0

    for i, det in enumerate(predictions):
        cls_name = det.get("class")
        if cls_name not in cls_to_fen: continue
        fen = cls_to_fen[cls_name]
        
        # แปลงเป็น Box (x1, y1, x2, y2)
        cx, cy, w, h = det["x"], det["y"], det["width"], det["height"]
        piece_box = (cx - w/2, cy - h/2, cx + w/2, cy + h/2)
        
        # *** KEY LOGIC 1: Piece Area ต้องคิดเฉพาะส่วนที่อยู่ใน Grid ***
        piece_area_in_grid = get_inter_area(piece_box, entire_board_box)
        
        # ถ้าหมากอยู่นอกกระดาน 100% ข้ามไปเลย
        if piece_area_in_grid <= 0:
            continue
            
        valid_pieces_count += 1
        
        # เอาหมากตัวนี้ไปทาบกับ 64 ช่อง
        for r in range(8):
            for c in range(8):
                sq_box = square_boxes[r][c]
                inter_area = get_inter_area(piece_box, sq_box)
                
                if inter_area > 0:
                    # *** KEY LOGIC 2: คำนวณ % โดยหารด้วย area ที่อยู่ใน grid ***
                    score = inter_area / piece_area_in_grid
                    
                    if score >= min_coverage_frac:
                        square_candidates[(r,c)].append({
                            'id': i,      # ID หมาก (เพื่อกันใช้ซ้ำ)
                            'fen': fen,
                            'score': score
                        })

    # --- 3. Loop Solving (เริ่มจากช่องที่มีผู้ท้าชิงคนเดียว) ---
    final_board = [["." for _ in range(8)] for _ in range(8)]
    used_piece_ids = set() # เก็บ ID หมากที่ถูกวางลงกระดานแล้ว
    
    while True:
        progress_made = False
        
        # A. คัดกรอง Candidates: เอาเฉพาะหมากที่ยังไม่ได้ถูกใช้ (not in used_piece_ids)
        current_state = {}
        for rc, cands in square_candidates.items():
            # ถ้าช่องนี้มีคนจองแล้ว ข้ามไป
            r, c = rc
            if final_board[r][c] != ".":
                continue
                
            valid_cands = [cand for cand in cands if cand['id'] not in used_piece_ids]
            if valid_cands:
                current_state[rc] = valid_cands

        if not current_state:
            break # ไม่มีช่องไหนเหลือให้แก้แล้ว

        # B. หาช่องที่มี "ผู้ท้าชิงคนเดียว" (Unique Candidate)
        solved_rc = None
        solved_cand = None
        
        # เรียงลำดับความง่าย: ช่องที่มี Candidate น้อยที่สุดก่อน (1 ตัว -> 2 ตัว -> ...)
        # ถ้าเท่ากัน ให้เอาช่องที่ Score สูงสุดก่อน
        sorted_squares = sorted(
            current_state.items(), 
            key=lambda x: (len(x[1]), -max(c['score'] for c in x[1]))
        )
        
        best_rc, best_cands = sorted_squares[0]
        
        if len(best_cands) == 1:
            # เจอช่องที่มีหมากตัวเดียวแย่งชิง -> ฟันธงเลย!
            solved_rc = best_rc
            solved_cand = best_cands[0]
        else:
            # ถ้าทุกช่องมีผู้ท้าชิงหลายคน (Deadlock)
            # ต้องเดา: เลือกคู่ที่มี Score สูงที่สุดในกระดานตอนนี้เพื่อ Break loop
            best_global_score = -1
            
            for rc, cands in current_state.items():
                # หาตัวที่คะแนนดีสุดในช่องนั้น
                top_cand = max(cands, key=lambda x: x['score'])
                if top_cand['score'] > best_global_score:
                    best_global_score = top_cand['score']
                    solved_rc = rc
                    solved_cand = top_cand

        # C. ลงบันทึก และ ตัดหมากออก
        if solved_rc and solved_cand:
            r, c = solved_rc
            final_board[r][c] = solved_cand['fen']
            used_piece_ids.add(solved_cand['id']) # หมากตัวนี้ถูกใช้แล้ว ห้ามไปโผล่ช่องอื่น
            progress_made = True
            # print(f"Assigned {solved_cand['fen']} (id {solved_cand['id']}) to {solved_rc} with score {solved_cand['score']:.2f}")

        if not progress_made:
            break # กัน Infinite loop (จริงๆ logic ข้อ else ข้างบนน่าจะกันได้แล้ว)

    return final_board


In [ ]:
def debug_overlay_grid_and_pieces(
    img_bgr: np.ndarray,
    grid: np.ndarray,
    pred_json: dict,
    out_path: Path = None,
    show: bool = True
):
    """
    Draw grid (from 9x9 intersections) + piece detections (from pred_json)
    on top of img_bgr for debugging.

    Args:
        img_bgr: board image (BGR, same coords as grid & preds)
        grid: (9,9,2) numpy array of intersection points (x,y)
        pred_json: Roboflow-style prediction JSON (with 'predictions' list)
        out_path: optional Path to save the debug image
        show: if True, display via matplotlib
    """
    vis = img_bgr.copy()

    # --- 1) Draw grid intersections as small circles ---
    for r in range(grid.shape[0]):
        for c in range(grid.shape[1]):
            x, y = grid[r, c]
            cv2.circle(vis, (int(x), int(y)), 3, (0, 255, 255), -1)  # yellow dots

    # --- 2) Draw grid lines (9 horizontal, 9 vertical) ---
    # horizontal lines
    for r in range(grid.shape[0]):
        pts = grid[r, :, :].astype(np.int32)
        cv2.polylines(vis, [pts], isClosed=False, color=(0, 255, 0),
                      thickness=2, lineType=cv2.LINE_AA)
    # vertical lines
    for c in range(grid.shape[1]):
        pts = grid[:, c, :].astype(np.int32)
        cv2.polylines(vis, [pts], isClosed=False, color=(0, 255, 0),
                      thickness=2, lineType=cv2.LINE_AA)

    # --- 3) Draw piece detections (center-x, center-y, width, height) ---
    for det in pred_json.get("predictions", []):
        cx = det["x"]
        cy = det["y"]
        w  = det["width"]
        h  = det["height"]
        cls_name = det.get("class", "?")
        conf = det.get("confidence", 0.0)

        x1 = int(cx - w / 2)
        y1 = int(cy - h / 2)
        x2 = int(cx + w / 2)
        y2 = int(cy + h / 2)

        cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 0, 255), 2)

        label = f"{cls_name} {conf:.2f}"
        cv2.putText(
            vis,
            label,
            (x1, max(0, y1 - 5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 255),
            1,
            cv2.LINE_AA
        )

    if out_path is not None:
        out_path = Path(out_path)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), vis)

    if show:
        plt.figure(figsize=(6, 10))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title("Grid + Piece Predictions Overlay")
        plt.show()

    return vis


In [ ]:
def image2grid(
    chess_img: np.ndarray,
    board_img: np.ndarray,
    offset_vec: np.ndarray,
    grid_out_dir: Path,
    chess_model,
    piece_confidence: int = 0.4,
    piece_overlap: int = 80,
    debug: bool = False
):
    grid_out_dir = Path(grid_out_dir)
    grid_out_dir.mkdir(parents=True, exist_ok=True)

    if board_img is None or chess_img is None:
         return None, None, None, None

    # 3) detect grid in board_img
    _, _, _, dots = detect_grid(board_img, grid_out_dir)

    # 4) intersections -> 9x9 grid (still in board_for_grid coords)
    grid_gridimg = dots_to_grid(dots, n_lines=N_GRID)  # N_GRID = 9

    # 4b) shift to chess_img coords
    grid_pieces = grid_gridimg + offset_vec.reshape(1, 1, 2)

    # 5) piece detection on chess_img (bigger padding)
    tmp_path = str(grid_out_dir / "board_for_pieces.jpg")
    cv2.imwrite(tmp_path, chess_img)

    pred_json = get_predictions_json(
            tmp_path,
            model=chess_model,
            confidence=piece_confidence,
            overlap=piece_overlap
    )

    # 6) build board matrix using grid in chess_img coords
    board_matrix = build_board_from_predictions(pred_json, grid_pieces)

    # 7) debug overlay MUST use chess_img + grid_pieces
    if debug :
        debug_path = grid_out_dir / "debug_overlay.jpg"
        debug_overlay_grid_and_pieces(
            img_bgr=chess_img,
            grid=grid_pieces,
            pred_json=pred_json,
            out_path=debug_path,
            show=True
        )

    return chess_img, grid_pieces, pred_json, board_matrix, grid_gridimg

## Flip

In [ ]:
def calculate_needed_rotations(board: list[list[str]]) -> int: 
    def count_white(rows_or_cols):
        count = 0
        for row_or_col in rows_or_cols:
            for piece in row_or_col:
                if piece.isupper():
                    count += 1
        return count

    # --- 1. Check Horizontal (Rows) ---
    # Rows 0 & 1 (Top) vs. Rows 6 & 7 (Bottom)
    white_at_top = count_white(board[:2])
    white_at_bottom = count_white(board[6:])

    # --- 2. Check Vertical (Columns) ---
    # Cols 0 & 1 (Left) vs. Cols 6 & 7 (Right)
    transposed = [list(row) for row in zip(*board)]
    white_at_left = count_white(transposed[:2])
    white_at_right = count_white(transposed[6:])

    # --- 3. Determine Dominant White Area ---
    if white_at_bottom >= max(white_at_top, white_at_left, white_at_right):
        return 0  # 0 rotations needed (Standard orientation)
    if white_at_left >= max(white_at_bottom, white_at_top, white_at_right):
        return 1  # 90 degrees (1 rotation)
    if white_at_top >= max(white_at_bottom, white_at_left, white_at_right):
        return 2  # 180 degrees (2 rotations)
    if white_at_right >= max(white_at_bottom, white_at_top, white_at_left):
        return 3  # 270 degrees (3 rotations)
    return 0

In [ ]:
def rotate_90_antiClockwise(board: list[list[str]]) -> list[list[str]]: 
        transposed = [list(row) for row in zip(*board)]
        rotated_board = transposed[::-1]
        return rotated_board

In [ ]:
def rotateAntiClockwise_x_times(board: list[list[str]], times: int) -> list[list[str]]:
    if times == 0:
        return board  
    
    current_board = board
    for _ in range(times):
        current_board = rotate_90_antiClockwise(current_board)
        
    return current_board

## Moved Matrix Construction

In [ ]:
def movable_board(cleaned_mask, grid_points, threshold_ratio=0.3, debug=False):
    if cleaned_mask is None:
        return np.ones((8, 8), dtype=bool)

    movement_matrix = np.zeros((8, 8), dtype=bool)
    
    # ถ้า Debug: สร้างภาพสีจาก Mask ขาวดำ เพื่อจะได้วาดเส้นสีทับได้
    if debug:
        debug_img = cv2.cvtColor(cleaned_mask, cv2.COLOR_GRAY2BGR)

    # วนลูปเช็คทีละช่อง
    for r in range(8):
        for c in range(8):
            # 1. ดึงพิกัด 4 มุม
            p1 = grid_points[r, c]
            p2 = grid_points[r, c+1]
            p3 = grid_points[r+1, c]
            p4 = grid_points[r+1, c+1]
            
            pts = np.array([p1, p2, p4, p3], dtype=np.int32)
            
            # 2. สร้างหน้ากากเฉพาะช่องนี้เพื่อคำนวณพื้นที่
            single_square_mask = np.zeros_like(cleaned_mask)
            cv2.fillPoly(single_square_mask, [pts], 255)
            
            square_area = cv2.countNonZero(single_square_mask)
            if square_area == 0: continue

            # 3. ตัดเอาเฉพาะส่วนที่ขยับ (Intersection)
            intersection = cv2.bitwise_and(cleaned_mask, cleaned_mask, mask=single_square_mask)
            move_count = cv2.countNonZero(intersection)
            
            # 4. คำนวณ Ratio
            current_ratio = move_count / square_area
            
            # เช็คว่าขยับหรือไม่
            is_moved = current_ratio > threshold_ratio
            movement_matrix[r, c] = is_moved

            # --- ส่วนแสดงผล Debug ---
            if debug:
                # สีแดง (Move), สีเขียว (Static)
                color = (255, 0, 0) if is_moved else (0, 255, 0) # ใน Matplotlib เป็น RGB (แดง, เขียว, น้ำเงิน)
                thickness = 2 if is_moved else 1
                
                # วาดกรอบสี่เหลี่ยมตาม Grid
                cv2.polylines(debug_img, [pts], isClosed=True, color=color, thickness=thickness)
                
                # เขียนค่า Ratio ลงไปกลางช่อง (จะได้รู้ว่าทำไมถึงผ่าน/ไม่ผ่าน)
                center_x = int((p1[0] + p4[0]) / 2)
                center_y = int((p1[1] + p4[1]) / 2)
                
                # ถ้า Ratio น้อยมาก ไม่ต้องเขียนเลขให้รก
                if current_ratio > 0.01:
                    cv2.putText(debug_img, f"{current_ratio:.2f}", (center_x-15, center_y+5), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 0), 1)

    if debug:
        plt.figure(figsize=(10, 10))
        plt.imshow(debug_img) # ไม่ต้องกลับสีเพราะเราทำ BGR -> RGB หรือวาดลงไปตรงๆ ถ้าใช้ cmap
        plt.title(f"Grid Overlay (Threshold: {threshold_ratio})\nRed = Moved, Green = Still")
        plt.axis("off")
        plt.show()

    return movement_matrix

## Majority Vote Recheck

In [ ]:
def unify_segment(boards, r, c, start_idx, end_idx, empty_symbol, debug):
    if start_idx > end_idx:
        return 0

    # 1. ดึงค่าดิบ (Raw Values)
    candidates = []
    for k in range(start_idx, end_idx + 1):
        candidates.append(boards[k][r, c])

    # 2. คำนวณสถิติ
    total_count = len(candidates)
    if total_count == 0: return 0

    non_empty = [x for x in candidates if x != empty_symbol]
    empty_count = total_count - len(non_empty)
    
    # คำนวณ % ของช่องว่าง
    empty_ratio = empty_count / total_count

    # 3. ตัดสินใจเลือกผู้ชนะ (Winner)
    final_val = empty_symbol
    
    # --- LOGIC ใหม่: เช็ค % Empty ก่อน ---
    if empty_ratio > 0.50:
        # ถ้ามีช่องว่างเกิน 40% -> ให้ผลลัพธ์เป็น "ว่าง" (Empty)
        final_val = empty_symbol
        decision_reason = f"Empty ratio {empty_ratio:.2f} > 0.40"
    elif len(non_empty) > 0:
        # ถ้าช่องว่างน้อยกว่า 40% (แปลว่ามีหมากหนาแน่นเกิน 60%) -> โหวตหาตัวหมาก
        counts = Counter(non_empty)
        final_val = counts.most_common(1)[0][0] 
        decision_reason = "Majority piece vote (Empty <= 40%)"
    else:
        # กรณีกันเหนียว (เช่น 0% empty แต่ไม่มี non_empty ซึ่งเป็นไปไม่ได้)
        final_val = empty_symbol
        decision_reason = "Fallback"

    # --- LOGIC การนับว่าแก้ไปกี่ตัว และ Debug ---
    fix_count = 0
    indices_fixed = []
    
    for k in range(start_idx, end_idx + 1):
        current_val = boards[k][r, c]
        if current_val != final_val:
            boards[k][r, c] = final_val
            fix_count += 1
            indices_fixed.append(k)

    # --- SHOW THINKING ---
    if debug and fix_count > 0:
        print(f"\n[Square {r},{c}] Segment: Frame {start_idx} to {end_idx}")
        print(f"Context: No movement detected in this segment.")
        
        timeline_str = "   Raw Timeline: "
        for idx, val in enumerate(candidates):
            frame_num = start_idx + idx
            if frame_num in indices_fixed:
                timeline_str += f"[{val}]* " 
            else:
                timeline_str += f"[{val}]  "
        print(timeline_str)
        
        # แสดงข้อมูลสถิติที่ใช้ตัดสินใจ
        print(f"   Stats: Total={total_count}, Empty={empty_count} ({empty_ratio*100:.1f}%)")
        
        if len(non_empty) > 0:
            vote_summary = str(dict(Counter(non_empty)))
            print(f"   Thinking: Found pieces {vote_summary}.")
        
        print(f"   Decision: Set to '{final_val}' because [{decision_reason}].")
        print(f"   Action: Fixed {fix_count} frames (indices {indices_fixed}) -> set to '{final_val}'")

    return fix_count

In [ ]:
def majorityVote_recheck(board_list, moved_list, empty_symbol='.', debug=True):
    """
    board_list: List ของ 8x8 boards
    moved_list: List ของ movement mask (0/1)
    debug: ถ้า True จะ print กระบวนการคิดออกมาให้ดูเฉพาะช่องที่มีการแก้ค่า
    """
    if not board_list or not moved_list:
        return board_list

    n_frames = len(board_list)
    refined_boards = [np.copy(b) for b in board_list]
    moves = [np.array(m, dtype=int) for m in moved_list]

    # ตัวแปรนับจำนวนการแก้ไข (เพื่อสรุปตอนท้าย)
    total_fixes = 0

    if debug:
        print("\n" + "="*60)
        print("MAJORITY RECHECK: THINKING PROCESS")
        print("="*60)

    for r in range(8):
        for c in range(8):
            # ดึงข้อมูลการขยับของช่องนี้ตลอด Timeline
            pixel_moves = [moves[i][r, c] for i in range(len(moves))]
            
            start_idx = 0
            for i, is_moved in enumerate(pixel_moves):
                if is_moved == 1:
                    # จบ Segment นิ่งช่วงแรก -> ทำการโหวต
                    end_idx = i
                    fixes = unify_segment(refined_boards, r, c, start_idx, end_idx, empty_symbol, debug)
                    total_fixes += fixes
                    start_idx = i + 1
            
            # ทำ Segment สุดท้าย
            fixes = unify_segment(refined_boards, r, c, start_idx, n_frames - 1, empty_symbol, debug)
            total_fixes += fixes

    if debug:
        print("-" * 60)
        print(f"Recheck Complete. Total values corrected: {total_fixes}")
        print("=" * 60 + "\n")

    return refined_boards

## Deduplication By Grid Similarity Check

In [ ]:
def dedup_by_grid_similarity_check(board_list):
    """
    Removes consecutive duplicate frames from a list of board states.
    Logic: If index 1 == index 2, ignore index 2. Then compare index 1 to index 3.
    """
    # 1. Handle edge cases (empty or single-frame lists)
    if not board_list:
        return []
    if len(board_list) == 1:
        return board_list

    # 2. Initialize the result list with the first frame
    deduplicated = [board_list[0]]

    # 3. Iterate through the rest of the list
    for i in range(1, len(board_list)):
        current_frame = board_list[i]
        last_kept_frame = deduplicated[-1]

        # Python's '!=' operator handles deep comparison for lists of lists automatically
        if not np.array_equal(current_frame, last_kept_frame):
           deduplicated.append(current_frame)

    print(f"Frames reduced from {len(board_list)} to {len(deduplicated)}")
    return deduplicated

## Cleaned Board Sequence

In [ ]:
def grid_to_board(grid):
    board = chess.Board()
    board.clear() # Start empty

    # The input grid usually starts at Row 0 (Rank 8) down to Row 7 (Rank 1)
    for row_idx, row in enumerate(grid):
        for col_idx, char in enumerate(row):
            if char != '.':
                piece = chess.Piece.from_symbol(char)
                # Convert grid (row, col) to chess square index
                # Rank 0 in grid is Rank 8 in chess (index 7)
                square = chess.square(col_idx, 7 - row_idx) 
                board.set_piece_at(square, piece)
    return board

In [ ]:
def board_to_grid(board):
    return [
        [
            (p.symbol() if (p := board.piece_at(chess.square(c, r))) else '.') 
            for c in range(8)
        ]
        for r in range(7, -1, -1)
    ]

In [ ]:
def boards_to_pgn_move(prev_board, next_board, override_turn=False, override_rule=False):
    # Detect the move by comparing squares
    move = None

    start_list = collections.deque()
    destination_list = []
    
    # Iterate over all 64 squares
    for square in chess.SQUARES:
        prev_piece = prev_board.piece_at(square)
        next_piece = next_board.piece_at(square)
        
        if prev_piece != next_piece:
            if prev_piece and next_piece is None:  # Piece moved FROM this square
                if prev_piece.piece_type == chess.KING:
                    start_list.appendleft((prev_piece, square))
                else:
                    start_list.append((prev_piece, square))
            elif next_piece:  # Piece moved TO this square
                if prev_piece is None: # Move to empty square
                    destination_list.append((next_piece, square, False))
                elif prev_piece.color != next_piece.color: # Capture
                    destination_list.append((next_piece, square, True))

    is_move = False
    out = ""
    misc = []

    for piece_from, pos_from in start_list:
        for piece_to, pos_to, is_capture in destination_list:
            
            # --- START: MODIFIED LOGIC ---
            candidate_move = None
            
            # กรณีที่ 1: หน้าตาเหมือนกัน (ปกติ)
            if piece_from == piece_to:
                candidate_move = chess.Move(pos_from, pos_to)
                
            # กรณีที่ 2: หน้าตาไม่เหมือนกัน (เช่น Promotion หรือ Bug) -> ลองเอาตัวหายเดินดู
            elif piece_from.piece_type != piece_to.piece_type:
                
                # เช็ค Promotion (เบี้ยเดินไปสุดกระดานแล้วเปลี่ยนร่าง)
                promotion_val = None
                if piece_from.piece_type == chess.PAWN:
                    if (piece_from.color == chess.WHITE and chess.square_rank(pos_to) == 7) or \
                       (piece_from.color == chess.BLACK and chess.square_rank(pos_to) == 0):
                        # ใช้ตัวที่ปลายทาง (piece_to) เป็นตัวกำหนด Promotion (เช่น =Q)
                        promotion_val = piece_to.piece_type
                
                # สร้าง Move จำลอง โดยใช้ promotion ถ้าจำเป็น
                test_move = chess.Move(pos_from, pos_to, promotion=promotion_val)
                
                # ถ้าเดินได้จริงตามกติกา (หรือบังคับ override) ให้ยอมรับ Move นี้
                if prev_board.is_legal(test_move) or override_rule:
                    candidate_move = test_move
            
            # ถ้าหา Move เจอ (ไม่ว่าจะแบบปกติ หรือแบบแก้ต่าง) ให้ดำเนินการต่อ
            if candidate_move is not None:
                move = candidate_move
                misc = [piece_from.color]
                
                # Turn override logic (crucial for state-based reconstruction)
                if override_turn:
                    prev_board.turn = piece_from.color
                
                # --- CRITICAL: SYNC NEXT BOARD TURN ---
                next_board.turn = not piece_from.color
                # --------------------------------------

                # Check for castling
                if prev_board.is_kingside_castling(move):
                    is_move = True
                    out = "O-O"
                    if next_board.is_check(): out += "+"
                    return is_move, out, misc
                
                elif prev_board.is_queenside_castling(move):
                    is_move = True
                    out = "O-O-O"
                    if next_board.is_check(): out += "+"
                    return is_move, out, misc

                # Validate and format move
                if prev_board.is_legal(move) or override_rule:
                    game = chess.pgn.Game()
                    game.setup(prev_board)
                    try:
                        game.add_main_variation(move)
                        is_move = True
                        # Extract the SAN (Standard Algebraic Notation)
                        out = str(game.variation(0)).split()[1]
                    except:
                        out = move.uci()
                        is_move = True
                    
                    # --- ADD CHECK (+) LOGIC ---
                    if next_board.is_check():
                        if not out.endswith('+') and not out.endswith('#'):
                            out += "+"
                    # ---------------------------

                    return is_move, out, misc
            # --- END: MODIFIED LOGIC ---

    return is_move, out, misc

In [ ]:
def clean_board_sequence(raw_states, debug=False):
    current_board = grid_to_board(raw_states[0])
    
    if len(raw_states) < 2:
        return raw_states

    detected_moves = [] 

    # ---------------------------------------------------------
    # PART 1: Loop หา Move และ "แก้" raw_states ให้สะอาด (Correction)
    # ---------------------------------------------------------
    for i in range(len(raw_states) - 1):
        prev_board = current_board
        
        # แปลงเฟรมถัดไปมาเทียบ (ภาพดิบ อาจมี Noise/Error)
        next_raw_board = grid_to_board(raw_states[i+1]) 
        
        # ตรวจจับการเดิน (ใช้ Logic ที่เราแก้ไว้รองรับตัวหาย/ตัวตามไม่เหมือนกัน)
        is_move, pgn_move, misc = boards_to_pgn_move(
            prev_board, next_raw_board, override_turn=True, override_rule=True
        )

        if is_move:
            moved_color = 'w' if misc[0] else 'b'
            
            # --- สร้างกระดานอุดมคติ (Ideal Board) ---
            ideal_next_board = prev_board.copy()
            try:
                # บังคับเดินหมากตาม PGN ที่ detect ได้ (ตัดปัญหาหมากผิดประเภท)
                ideal_next_board.push_san(pgn_move)
                
                # เขียนทับ raw_states ด้วยข้อมูลที่ถูกต้อง 100%
                raw_states[i+1] = board_to_grid(ideal_next_board)
                
                # อัปเดต current_board เพื่อใช้เทียบในรอบถัดไป
                current_board = ideal_next_board
                
                # if debug:
                #     print(f"Frame {i}->{i+1}: Move {pgn_move} ({moved_color}) >> Fixed & Saved.")

            except ValueError:
                # กรณี Move พังจริงๆ (Fallback) ให้ใช้ภาพดิบไปก่อน
                current_board = next_raw_board
                # if debug: print(f"Frame {i}->{i+1}: Error pushing move, using raw board.")

            detected_moves.append({'color': moved_color, 'idx': i+1})
            
        else:
            # --- กรณีไม่ขยับ (Noise Reduction) ---
            # บังคับให้ raw_states[i+1] เหมือนกับภาพล่าสุด (current_board)
            # เพื่อลบ Noise (เช่น มือคนบัง, แสงวูบวาบ) ในเฟรมที่ไม่มีการเดิน
            raw_states[i+1] = board_to_grid(current_board)
            pass

    # ---------------------------------------------------------
    # PART 2: Filter Logic (กรองเอาเฉพาะเฟรมสุดท้ายของแต่ละสี)
    # ---------------------------------------------------------
    cleaned_states = [raw_states[0]] # เก็บภาพแรกเสมอ
    n_moves = len(detected_moves)
    
    if debug:
        print("-" * 40)
        print(f"Total Moves Found: {n_moves}")
        print("-" * 40)

    for i in range(n_moves):
        current_move = detected_moves[i]
        curr_color = current_move['color']
        real_idx = current_move['idx']
        
        keep = False
        reason = ""

        # เงื่อนไขการเก็บ: ตัวสุดท้าย หรือ ตัวที่สีเปลี่ยนในตาถัดไป
        if i == n_moves - 1:
            keep = True
            reason = "Last Move"
        else:
            next_color = detected_moves[i+1]['color']
            if curr_color != next_color:
                keep = True # สีเปลี่ยน (เช่น ขาวเสร็จแล้ว ตาต่อไปเป็นดำ)
                reason = f"Switch ({curr_color}->{next_color})"
            else:
                keep = False # สีเดิม (เป็น state ระหว่างอนิเมชั่น)

        if debug:
            status = "✅ KEEP" if keep else "❌ DROP"
            print(f"Move #{i+1} (Frame {real_idx}): {curr_color} | {status} | {reason}")
            print_board(raw_states[real_idx])

        # เก็บภาพจริง (ที่ผ่านการ Clean ใน Part 1 แล้ว)
        if keep:
            cleaned_states.append(raw_states[real_idx])

    if debug: print("="*40 + "\n")

    return cleaned_states

## Board2PGN

In [ ]:
def raw_states_to_pgn(board_states):
    # Convert all raw grids to chess.Board objects
    boards = [grid_to_board(state) for state in board_states]
    
    pgn_movelist = ""
    move_number = 1
    
    # We need at least 2 boards to make a move
    if len(boards) < 2:
        return ""

    for i in range(len(boards) - 1):
        prev = boards[i]
        curr = boards[i+1]
        
        # Use your existing logic logic with overrides enabled
        is_move, pgn_move, misc = boards_to_pgn_move(
            prev, curr, override_turn=True, override_rule=True
        )

        if is_move:
            moved_color = misc[0] # chess.WHITE (True) or chess.BLACK (False)
            
            # Format the PGN string based on color
            if moved_color == chess.WHITE:
                pgn_movelist += f"{move_number}. {pgn_move} "
            else:
                # If Black moves, check if we need "1..." (start of game) or just append
                if i == 0: 
                    pgn_movelist += f"{move_number}... {pgn_move} "
                else:
                    pgn_movelist += f"{pgn_move} "
                # Increment move number after Black's move
                move_number += 1
                
    return pgn_movelist.strip()

## Evaluation 

In [ ]:
ground_truth = {
    "2" : [
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'N'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['n', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'P', '.', '.', 'P', '.', '.', '.'],
            ['.', '.', 'N', '.', '.', '.', '.', '.'],
            ['P', '.', 'P', '.', '.', 'K', 'P', 'P'],
            ['R', '.', '.', 'Q', '.', 'B', '.', 'R']
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', 'n', 'N'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['n', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'P', '.', '.', 'P', '.', '.', 'q'],
            ['.', '.', 'N', '.', '.', '.', '.', '.'],
            ['P', '.', 'P', '.', '.', 'K', 'P', 'P'],
            ['R', '.', '.', 'Q', '.', 'B', '.', 'R']
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', 'n', 'N'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['n', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'P', '.', '.', 'P', '.', '.', 'q'],
            ['.', '.', 'N', '.', '.', '.', 'P', '.'],
            ['P', '.', 'P', '.', '.', 'K', '.', 'P'],
            ['R', '.', '.', 'Q', '.', 'B', '.', 'R']
        ]
    ],
    "4" : [
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', 'p', '.', '.'],
            ['n', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'P', '.', '.', 'P', '.', '.', 'N'],
            ['.', '.', 'N', '.', '.', '.', 'B', '.'],
            ['P', '.', 'P', '.', '.', 'P', 'P', 'P'],
            ['R', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['n', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'P', '.', '.', 'P', 'p', '.', 'N'],
            ['.', '.', 'N', '.', '.', '.', 'B', '.'],
            ['P', '.', 'P', '.', '.', 'P', 'P', 'P'],
            ['R', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', 'N', '.'],
            ['n', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'P', '.', '.', 'P', 'p', '.', '.'],
            ['.', '.', 'N', '.', '.', '.', 'B', '.'],
            ['P', '.', 'P', '.', '.', 'P', 'P', 'P'],
            ['R', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', 'N', '.'],
            ['n', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'P', '.', '.', 'P', '.', '.', '.'],
            ['.', '.', 'N', '.', '.', '.', 'p', '.'],
            ['P', '.', 'P', '.', '.', 'P', 'P', 'P'],
            ['R', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'N'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['n', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'P', '.', '.', 'P', '.', '.', '.'],
            ['.', '.', 'N', '.', '.', '.', 'p', '.'],
            ['P', '.', 'P', '.', '.', 'P', 'P', 'P'],
            ['R', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ]
    ],
    "6" : [
        [
            ['.', 'r', '.', '.', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'b', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['P', 'N', '.', 'P', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', 'P', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['P', '.', 'P', '.', '.', '.', '.', 'P'],
            ['.', 'R', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['.', 'r', '.', '.', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['P', 'b', '.', 'P', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', 'P', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['P', '.', 'P', '.', '.', '.', '.', 'P'],
            ['.', 'R', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['.', 'r', '.', '.', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', '.', '.', 'p', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['P', 'R', '.', 'P', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', 'P', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['P', '.', 'P', '.', '.', '.', '.', 'P'],
            ['.', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['.', 'r', '.', '.', 'k', 'b', 'n', 'r'],
            ['p', '.', 'p', '.', '.', '.', 'p', '.'],
            ['.', 'p', '.', 'p', '.', '.', '.', '.'],
            ['P', 'R', '.', 'P', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', 'P', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['P', '.', 'P', '.', '.', '.', '.', 'P'],
            ['.', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['.', 'r', '.', '.', 'k', 'b', 'n', 'r'],
            ['p', '.', 'p', '.', '.', '.', 'p', '.'],
            ['.', 'p', '.', 'p', '.', '.', '.', '.'],
            ['P', 'R', '.', 'P', 'p', '.', '.', '.'],
            ['.', '.', 'P', '.', 'P', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['P', '.', '.', '.', '.', '.', '.', 'P'],
            ['.', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['.', 'r', '.', '.', 'k', 'b', '.', 'r'],
            ['p', '.', 'p', '.', 'n', '.', 'p', '.'],
            ['.', 'p', '.', 'p', '.', '.', '.', '.'],
            ['P', 'R', '.', 'P', 'p', '.', '.', '.'],
            ['.', '.', 'P', '.', 'P', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['P', '.', '.', '.', '.', '.', '.', 'P'],
            ['.', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
        [
            ['.', 'r', '.', '.', 'k', 'b', '.', 'r'],
            ['p', '.', 'p', '.', 'n', '.', 'p', '.'],
            ['.', 'p', '.', 'p', '.', '.', '.', '.'],
            ['P', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', '.', 'P', '.', 'P', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', '.', '.', 'P'],
            ['.', '.', '.', 'Q', 'K', 'B', '.', 'R']
        ],
    ],
    "8" : [
        [
            ['.', 'r', '.', '.', 'k', '.', '.', 'q'],
            ['p', '.', 'p', '.', 'n', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', 'Q', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', 'r', '.', '.', 'k', '.', '.', 'q'],
            ['p', '.', 'p', '.', 'n', '.', '.', '.'],
            ['.', '.', '.', 'p', 'Q', '.', 'p', 'b'],
            ['P', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', 'r', '.', 'k', '.', '.', '.', 'q'],
            ['p', '.', 'p', '.', 'n', '.', '.', '.'],
            ['.', '.', '.', 'p', 'Q', '.', 'p', 'b'],
            ['P', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', 'r', '.', 'k', '.', '.', '.', 'q'],
            ['p', '.', 'p', '.', 'n', 'Q', '.', '.'],
            ['.', '.', '.', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', 'r', '.', 'k', '.', '.', '.', 'q'],
            ['p', '.', '.', '.', 'n', 'Q', '.', '.'],
            ['.', '.', 'p', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', 'r', '.', 'k', '.', '.', '.', 'q'],
            ['p', '.', '.', '.', 'n', '.', '.', '.'],
            ['.', '.', 'p', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'P', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', 'Q', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', 'r', '.', 'k', '.', '.', '.', 'q'],
            ['p', '.', '.', '.', 'n', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'p', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', 'Q', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', 'r', '.', 'k', '.', '.', '.', 'q'],
            ['Q', '.', '.', '.', 'n', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'p', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', '.', 'r', 'k', '.', '.', '.', 'q'],
            ['Q', '.', '.', '.', 'n', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'p', 'p', '.', '.', '.'],
            ['.', 'p', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', 'R', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', '.', 'r', 'k', '.', '.', '.', 'q'],
            ['Q', '.', '.', '.', 'n', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'p', 'p', '.', '.', '.'],
            ['.', 'R', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', '.', 'r', 'k', '.', '.', '.', 'q'],
            ['Q', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', 'n', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'p', 'p', '.', '.', '.'],
            ['.', 'R', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ],
        [
            ['.', '.', 'r', 'k', '.', '.', '.', 'q'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', 'Q', 'n', 'p', '.', '.', 'p', 'b'],
            ['P', '.', '.', 'p', 'p', '.', '.', '.'],
            ['.', 'R', 'P', '.', 'P', '.', '.', 'P'],
            ['.', '.', '.', 'B', '.', '.', 'P', '.'],
            ['P', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'K', 'R', '.', '.']
        ]
    ],
    "B" : [
        [
            ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
            ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R'],
        ],
        [
            ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R'],
        ],
        [
            ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', 'p', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R'],
        ],
        [
            ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', 'p', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', 'N', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', 'N', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', 'n', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', 'p', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', 'N', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', 'N', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', 'n', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', 'p', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', 'N', '.', '.', 'N', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', 'p', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', '.', '.', '.', '.'],
            ['.', '.', '.', 'p', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', 'N', '.', '.', 'N', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', 'p', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', '.', '.', '.', '.'],
            ['.', '.', '.', 'N', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', 'N', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', 'p', '.', '.', '.'],
            ['.', '.', '.', 'N', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', 'N', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', 'N', 'N', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', 'q'],
            ['.', '.', '.', '.', 'N', 'N', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', 'N'],
            ['.', '.', '.', '.', 'N', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', 'p', '.', '.', 'n'],
            ['.', '.', '.', '.', '.', 'b', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', 'N'],
            ['.', '.', '.', '.', 'N', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', 'p', '.', '.', 'n'],
            ['.', '.', '.', '.', '.', 'N', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', 'N', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'n', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', 'N', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', 'p', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'N', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'p', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'n', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'p', '.', '.'],
            ['.', '.', '.', 'P', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'p', '.', '.'],
            ['.', '.', '.', 'n', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', 'Q', 'K', 'B', '.', 'R'],
        ],
        [
            ['r', '.', '.', '.', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'p', '.', '.'],
            ['.', '.', '.', 'Q', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'p', '.', '.'],
            ['.', '.', '.', 'Q', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', 'k', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'Q', 'p', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', '.', 'b', '.', 'r'],
            ['p', 'p', 'p', 'k', '.', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', 'Q', 'p', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', '.', 'b', '.', 'r'],
            ['p', 'p', 'p', 'k', '.', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'Q', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', '.', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'p', 'p', 'p'],
            ['.', '.', 'k', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'Q', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', '.', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'Q', 'p', 'p'],
            ['.', '.', 'k', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', '.', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'Q', '.', 'p'],
            ['.', '.', 'k', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'p', '.'],
            ['.', '.', '.', '.', '.', 'P', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', '.', 'b', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'Q', '.', 'p'],
            ['.', '.', 'k', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', '.', '.', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'Q', '.', 'p'],
            ['.', '.', 'k', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['.', 'b', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
        [
            ['.', '.', '.', 'r', '.', '.', '.', 'r'],
            ['p', 'p', 'p', '.', '.', 'Q', '.', 'p'],
            ['.', '.', 'k', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', 'P', '.'],
            ['.', 'b', '.', '.', '.', '.', '.', '.'],
            ['.', '.', 'P', '.', '.', '.', '.', '.'],
            ['P', 'P', '.', '.', 'P', '.', 'P', 'P'],
            ['R', '.', 'B', '.', 'K', 'B', '.', 'R'],
        ],
    ]
}

In [ ]:
def chess_detect_eval(num: int, pred: list):
    if num not in ground_truth:
        return {"success": False, "errors": f"Key {num} not found in ground_truth."}
    
    gt_seq = ground_truth[num]
    errors = []
    
    # length mismatch
    if len(pred) != len(gt_seq):
        errors.append({
            "type": "step_length_mismatch",
            "expected": len(gt_seq),
            "got": len(pred)
        })

    for step in range(len(pred)):
        pred_board = pred[step]
        gt_board = gt_seq[step]
        
        board_mismatches = []
        
        # Check board dimensions
        if len(pred_board) != 8 or any(len(row) != 8 for row in pred_board):
            errors.append({
                "step": step,
                "type": "invalid_board_dimension",
                "message": "Board must be 8x8"
            })
            continue

        # Check cell by cell
        for row in range(8):
            for col in range(8):
                if pred_board[row][col] != gt_board[row][col]:
                    board_mismatches.append({
                        "position": (row, col),
                        "expected": gt_board[row][col],
                        "got": pred_board[row][col]
                    })
        
        if board_mismatches:
            errors.append({
                "state": step,
                "type": "content_mismatch",
                "mismatches": board_mismatches
            })
            
    success = len(errors) == 0
    return {
        "success": success,
        "errors": errors
    }

In [ ]:
def show_error(result: dict):
    if result.get("success", False):
        print("its 100% True")
        return
        
    errors = result.get("errors", [])

    if isinstance(errors, str):
        print(f"Critical Error: {errors}")
        return

    print(f"--------------Prediction not match with Truth--------------")

    # 3. Iterate through error list
    for error in errors:
        error_type = error.get("type")

        # --- Case A: Sequence Length Mismatch ---
        if error_type == "step_length_mismatch":
            print(f"SEQUENCE LENGTH ERROR")
            print(f"   Expected {error['expected']} steps, but got {error['got']} steps.")
            print("-" * 30)

        # --- Case B: Invalid Board Dimension ---
        elif error_type == "invalid_board_dimension":
            step_idx = error.get("step")
            print(f"DIMENSION ERROR at Step {step_idx}")
            print(f"   {error.get('message')}")
            print("-" * 30)

        # --- Case C: Content Mismatch (The specifics you asked for) ---
        elif error_type == "content_mismatch":
            step_idx = error.get("state")
            mismatches = error.get("mismatches", [])
            
            print(f"MISMATCH AT STEP {step_idx}")
            print(f"   (Found {len(mismatches)} specific errors)")
            
            for m in mismatches:
                row, col = m['position']
                expected_val = m['expected']
                got_val = m['got']
                
                print(f"   • Row {row}, Col {col} | Expected: '{expected_val}' vs Got: '{got_val}'")
            
            print("-" * 30)

        else:
            print(f"Unknown Error Type: {error}")

## PipeLine 

In [ ]:
def video_to_boards_and_pgn(
    video_name: str, # name
    rela_path: str, # path
    hand_split_dir: str, # where to save image from video2image
    work_root: Path, # where to save all process result
    chess_model, # chess piece detection model
    hand_model, # hand detection model
    piece_confidence: int = 0.4, # chess piece detection confidence threshold
    piece_overlap: int = 80, # ratio if chess piece overlap > 80% consider as duplicate
    run_again: bool = False, # Is it last video
    debug: bool = False,
    grid_padding=40,
    image_padding=120, 
    skin_pixel_ts=500, # skin threshold for hand detection (Unit : pixel count)
    pixel_diff_ts=3000, # pixel difference threshold for deduplication (Unit : pixel count)
    moved_ratio_ts=0.05 # ratio threshold for moved piece detection (Unit : ratio of moved pixel to grid pixel)
):
    hand_split_dir = Path(hand_split_dir)
    work_root = Path(work_root)

    # Clear old frames so we only have this video's frames
    if not run_again :
        if hand_split_dir.exists():
            shutil.rmtree(hand_split_dir)
        hand_split_dir.mkdir(parents=True, exist_ok=True)
        
        work_root.mkdir(parents=True, exist_ok=True)
        video_path     = f"{rela_path}{video_name}.mp4"
        # 1) Video to image using YOLO hand detection model
        video2image(video_path = str(video_path), hand_model=hand_model, output_folder=str(hand_split_dir), debug=debug)

    # 2) Collect the frames that correspond to stable clean states 
    all_jpgs = sorted(hand_split_dir.glob("*.jpg"))
    clean_start_frames = [p for p in all_jpgs if "START_clean" in p.name]
    if not clean_start_frames:
        print("No START_clean frames found; check video or hand detector.")
        return [], ""

    # 3) Phase1 : process each frame -> Crop, Hand_detection, Deduplication
    print("--- Phase 1: Cropping, Cleaning, Deduplicating Frames ---")
    unique_tasks = []  # save as tuple (chess_img, board_img, offset, original_idx)
    last_valid_board_img = None

    for idx, frame_path in enumerate(clean_start_frames):
        frame_bgr = cv2.imread(str(frame_path))
        if frame_bgr is None:
            print(f"Warning: could not read frame {frame_path}")
            continue
        # Get cropped image 
        board_img, chess_img, offset_vec = get_roi( 
            frame_bgr,
            grid_padding=grid_padding,
            image_padding=image_padding
        )
        if board_img is None:
            continue
        # Detect hand by skin color pixel count
        hand_detected = hand_detection(board_img, cut_bound = grid_padding, skin_threshold = skin_pixel_ts, debug = debug)
        if hand_detected:
            continue
        # Check duplicate frame by pixel difference
        is_duplicate, moved_mask = dedup_by_pixel_similarity_check( # moved mask เทียบกับภาพก่อนหน้า
            board_img, 
            last_valid_board_img, 
            threshold=pixel_diff_ts, 
            debug=debug,  
            cut_bound = grid_padding
        )
        if is_duplicate:
            continue 
        # Store cleaned frame info
        unique_tasks.append({
            "chess_img": chess_img,
            "board_img": board_img,
            "offset_vec": offset_vec,
            "original_idx": idx,
            "moved_mask": moved_mask
            })
        last_valid_board_img = board_img

    print(f"Phase 1 Complete. Found {len(unique_tasks)} unique frames from {len(clean_start_frames)} total.")
    #Show Result of Phase 1
    if len(unique_tasks) == 0:
        print("No unique frames to show.")
    else:
        convertWith = cv2.COLOR_BGR2RGB
        for i, task in enumerate(unique_tasks):
            show_image(task["board_img"],f"Unique #{i+1} (Original Frame {task['original_idx']})", convertWith=convertWith)

    # 4) Phase2 : Process each frame -> Chess_Piece detection, Board_Matrix & Move_Matrix construction, Flip decision
    print("--- Phase 2: Chess_Piece detection, Board_Matrix & Move_Matrix construction, Flip decision---")
    board_list = []
    flip_for_white_bottom = None 
    moved_list = []
    for i, task in enumerate(unique_tasks):
        idx = task["original_idx"]
        
        # save_folder for this frame
        grid_out_dir = work_root / f"pos_{idx:02d}"
        grid_out_dir.mkdir(parents=True, exist_ok=True)
        

        try: 
            # Take board_img and chess_img to construct grid
            _, _, _, board_matrix, grid_gridimg = image2grid(
                chess_img=task["chess_img"],
                board_img=task["board_img"],
                offset_vec=task["offset_vec"],
                grid_out_dir=grid_out_dir,
                chess_model=chess_model,
                piece_confidence=piece_confidence,
                piece_overlap=piece_overlap,
                debug=debug
            )
            if debug:
                print_board(board_matrix)
            # Flip board to have white pieces at bottom
            if flip_for_white_bottom is None: # How many time to flup
                flip_for_white_bottom = calculate_needed_rotations(board_matrix)
                print(f"Need to flip: {flip_for_white_bottom} times")
            board_matrix = rotateAntiClockwise_x_times(board_matrix, flip_for_white_bottom)
            board_list.append(board_matrix)
            # Get moved_matrix
            moved_matrix = None
            if i > 0 and task["moved_mask"] is not None:
                moved_matrix = movable_board(task["moved_mask"], grid_gridimg, threshold_ratio=moved_ratio_ts,debug=debug)
                print_board(moved_matrix.astype(int)) # moved_matrix
                # Rotate moved_matrix to match board orientation
                moved_matrix = rotateAntiClockwise_x_times(moved_matrix, flip_for_white_bottom)
                moved_list.append(moved_matrix)
            
        except Exception as e:
            print(f"[Frame {idx}] Error processing: {e}")

    board_list = majorityVote_recheck(board_list, moved_list)
    board_list = dedup_by_grid_similarity_check(board_list)
    board_list = clean_board_sequence(board_list,debug)
    if not board_list:
        print("No boards successfully reconstructed.")
        return [], ""
    if debug:
        show_error(chess_detect_eval(video_name[0],board_list))   
    # --- 4) Convert board sequence to PGN ---
    pgn_str = raw_states_to_pgn(board_list)

    return board_list, pgn_str


## Execute

In [ ]:
# run_again = False

In [ ]:
# video_name = "2_Move_rotate_student"
# rela_path     = f"/kaggle/input/cu-chess-detection-2025/Chess Detection Competition/test_videos/"
# hand_split_dir = "/kaggle/working/clean_frames"
# work_root      = Path("/kaggle/working/fen_pipeline_validation")

# boards, pgn = video_to_boards_and_pgn(
#     video_name=video_name,
#     rela_path=rela_path,
#     hand_split_dir=hand_split_dir,
#     work_root=work_root,
#     piece_model=chess_model,       
#     piece_confidence=0.4,
#     piece_overlap=80,
#     run_again = True,
#     debug = True
# )
# run_again = True
# print("Number of board states:", len(boards))
# print(f"Temp PGN:\n{pgn}")
# print(raw_states_to_pgn(ground_truth[video_name[0]]))

In [ ]:
## fully run
import pandas as pd
csv_path = '/kaggle/input/cu-chess-detection-2025/Chess Detection Competition/sample-submission.csv'
df = pd.read_csv(csv_path)

rela_path     = f"/kaggle/input/cu-chess-detection-2025/Chess Detection Competition/test_videos/"
hand_split_dir = "/kaggle/working/clean_frames"
work_root      = Path("/kaggle/working/fen_pipeline_validation")

limit = min(5, len(df))  # Safety check in case df has fewer than 5 rows

for i in range(limit):
    video_name = df.iloc[i, 0][:-4]
    
    print(f"[{i+1}/{limit}] Processing: {video_name}...")
        # Call detection function
    try : 
        boards, pgn = video_to_boards_and_pgn(
                video_name=video_name,
                rela_path=rela_path,           # Ensure defined
                hand_split_dir=hand_split_dir, # Ensure defined
                work_root=work_root,           # Ensure defined
                chess_model=chess_model,       # Ensure defined
                hand_model=hand_model,         # Ensure defined
                piece_confidence=0.4,
                piece_overlap=80,
                run_again=False,
                debug=False,
                grid_padding=40,
                image_padding=120,
                skin_pixel_ts=0.00056, # ratio of 500 pixels
                pixel_diff_ts=0.0033, #ratio of 3000 pixels
                moved_ratio_ts=0.05
        )
        df.iloc[i, 1] = pgn
    except Exception as e:
        print(f"❌ Error on row {i}: {e}")
        # df.iloc[i, 1] = "ERROR" # Mark the CSV so you know it failed

print("\nUpdated DataFrame (First 5 rows):")
for i in range(limit):
    print(df.iloc[i, 1])

In [ ]:
for i in range(limit):
    video_name = df.iloc[i, 0][:-4]
    gt = raw_states_to_pgn(ground_truth[video_name[0]])
    print(gt)
    print(f"{video_name} is {gt == df.iloc[i,1]}")

In [ ]:
video_name = "Bonus Long Video Label"
rela_path     = f"/kaggle/input/cu-chess-detection-2025/Chess Detection Competition/bonus_video/"
hand_split_dir = "/kaggle/working/clean_frames"
work_root      = Path("/kaggle/working/fen_pipeline_validation")

boards, pgn = video_to_boards_and_pgn(
        video_name=video_name,
        rela_path=rela_path,
        hand_split_dir=hand_split_dir,
        work_root=work_root,
        chess_model=chess_model,
        hand_model=hand_model,
        piece_confidence=0.4,
        piece_overlap=80,
        run_again = False,
        debug= False,
        grid_padding=40,
        image_padding=120,
        skin_pixel_ts=0.00056, 
        pixel_diff_ts=0.0033,
        moved_ratio_ts=0.05
    )
print("Number of board states:", len(boards))
print("Temp PGN:\n", pgn)
df.iloc[5, 1] = pgn

In [ ]:
gt = raw_states_to_pgn(ground_truth['B'])
print(gt)
print(f"{video_name} is {gt == df.iloc[5,1]}")

In [ ]:
df.to_csv('submission.csv', index=False)

print("Successfully saved 'submission.csv'")